# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [ ]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-232bad8271-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Sport Workflow Series ("Association Football Club explorative search") 

Consider the following exploratory information need:

> investigate the association football clubs and find the main BGPs related to this sport. Compare clubs in countries and championships won. Explore also the participation of the countries in the FIFA World Cup.

**Important consideration: in wikidata, we are talking of ASSOCIATION FOOTBALL**

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q11571`| Cristiano Ronaldo       | node |





Also consider

```
wd:Q11571 ?p ?obj .
```

is the BGP to retrieve all **properties of Cristiano Ronaldo**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.



The workload should:


1. Identify the BGP for association football club

2. Identify the BGP for association football league

3. Return for each north american country the number of association football clubs

4. Identify the BGP for the UEFA Champions League.

5. Consider the countries which have at least two different clubs of their nationality who won the UEFA Champions League. Return the list of these countries with the number of different clubs who won the UEFA Champions League (the result set must be triples of country IRI, label and #clubs).

6. Return for each european country the top level league (the result set must be a list of country IRI and label, and league IRI and label).

7. Identify the BGP of the FIFA World Cup

8. Consider countries who participated at the FIFA World Cup more than 10 times. Return these countries with the number of apparitions (the result set must be triples of country IRI, label and #apparitions).

9. Consider the countries that won the FIFA World Cup at least twice. Find the association football clubs of those countries won the UEFA Champions League (the result set must be couples of clubs IRI and label).


## Task 1
Identify the BGP for association football club

In [9]:
# query example that retrieve all the properties of Cristiano Ronaldo
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q11571 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1005'), ('name', 'Portuguese National Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('name', 'native language')]
[('p', 'http://www.wikidata.org/prop/direct/P1038'), ('name', 'relative')]
[('p', 'http://www.wikidata.org/prop/direct/P1050'), ('name', 'medical condition')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P109'), ('name', 'signature')]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league')]
[('p', 'http://www.wikidata.org/prop/direct/P1263'), ('name', 'NNDB people ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1266'), ('name', 'AlloCiné person ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1273'), ('name', 'CANTIC ID')]
[('p', 'http:

In [7]:
# Since on the properties there's no reference to any association football,
# I'll try to retrieve all the objects of Cristiano Ronaldo
# MISTAKE: printed the properties URI
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q11571 ?p ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 150
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'FWA Footballer of the Year')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('name', "PFA Fans' Player of the Year")]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'Liga Portugal')]
[('p', 'http://www.wikidata.org/prop/direct/P735'), ('name', 'Cristiano')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'Commander of the Order of Merit of Portugal')]
[('p', 'http://www.wikidata.org/prop/direct/P54'), ('name', 'Portugal Olympic football team')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'UEFA Euro 2020')]
[('p', 'http://www.wikidata.org/prop/direct/P734'), ('name', 'dos Santos')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'UEFA Team of the Year')]
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('name', 'FIFA World Player of the Year')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'FIFA World Player of the Year')]
[('p', 'http:

In [10]:
# Since on the properties there's no reference to any association football,
# I'll try to retrieve all the objects of Cristiano Ronaldo
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q11571 ?p ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 150
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q1750244'), ('name', 'FWA Footballer of the Year')]
[('obj', 'http://www.wikidata.org/entity/Q10726649'), ('name', "PFA Fans' Player of the Year")]
[('obj', 'http://www.wikidata.org/entity/Q182994'), ('name', 'Liga Portugal')]
[('obj', 'http://www.wikidata.org/entity/Q12899500'), ('name', 'Cristiano')]
[('obj', 'http://www.wikidata.org/entity/Q20888805'), ('name', 'Commander of the Order of Merit of Portugal')]
[('obj', 'http://www.wikidata.org/entity/Q3590758'), ('name', 'Portugal Olympic football team')]
[('obj', 'http://www.wikidata.org/entity/Q373501'), ('name', 'UEFA Euro 2020')]
[('obj', 'http://www.wikidata.org/entity/Q20819738'), ('name', 'dos Santos')]
[('obj', 'http://www.wikidata.org/entity/Q1146578'), ('name', 'UEFA Team of the Year')]
[('obj', 'http://www.wikidata.org/entity/Q182529'), ('name', 'FIFA World Player of the Year')]
[('obj', 'http://www.wikidata.org/entity/Q260117'), ('name', "UEFA Men's Player of the Year Award"

In [11]:
# Now I'll search for Portugal national association football team Q267245 properties, I'll try to reach association football club
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q267245 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 150
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P3441'), ('name', 'FIFA country code')]
[('p', 'http://www.wikidata.org/prop/direct/P127'), ('name', 'owned by')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')]
[('p', 'http://www.wikidata.org/prop/direct/P1352'), ('name', 'ranking')]
[('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P2002'), ('name', 'Twitter username')]
[('p', 'http://www.wikidata.org/prop/direct/P2003'), ('name', 'Instagram username')]
[('p', 'http://www.wikidata.org/prop/direct/P2013'), ('name', 'Facebook ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2094'), ('name', 'competition class')]
[('p', 'http://www.wikidata.org/prop/direct/P2184'), ('name', 'history of topic')]
[('p', 'http://www.wikidata.org/prop/dir

In [12]:
# Now I'll search for Portugal national association football team Q267245 properties, I'll try to reach association football club

#looking for subclassof
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q267245 wdt:P279 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 150
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q6979593'), ('name', 'national association football team')]
1


In [13]:

#looking for instanceof
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q267245 wdt:P31 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 150
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q6979593'), ('name', 'national association football team')]
1


In [14]:
#Q6979593
#looking for instanceof
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q6979593 wdt:P279 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 150
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q15944511'), ('name', 'association football team')]
[('obj', 'http://www.wikidata.org/entity/Q1194951'), ('name', 'national sports team')]
2


In [15]:
#Q15944511
#looking for instanceof
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q15944511 wdt:P279 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 150
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q28083137'), ('name', 'football team')]
1


In [16]:
#Q15944511
#looking for instanceof
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q15944511 wdt:P31 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 150
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [17]:
# Try looking at Juventus Q1422 subclassof
#looking for instanceof
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q1422 wdt:P279 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 150
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [18]:
# Try looking at Juventus Q1422 properties
#looking for instanceof
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q1422 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 150
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P112'), ('name', 'founded by')]
[('p', 'http://www.wikidata.org/prop/direct/P115'), ('name', 'home venue')]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league')]
[('p', 'http://www.wikidata.org/prop/direct/P127'), ('name', 'owned by')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('name', 'named after')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1454'), ('name', 'legal form')]
[('p', 'http://www.wikidata.org/prop/direct/P154'), ('name', 'logo image')]
[('p', 'http://www.wikidata.org/prop/direct/P159'), ('name', 'headquarters location')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'award received')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P1830'), ('

In [19]:
# try looking at all object of association football P641
queryString = """
SELECT DISTINCT ?sName ?objName
WHERE {
   # bind something
   ?s wdt:P641 ?obj .
   # get the label
   ?s sc:name ?sName.
   ?obj sc:name ?objName.
}
LIMIT 150
"""

print("Results")
x=run_query(queryString)

Results
[('sName', 'sports in Sweden'), ('objName', 'cycling')]
[('sName', '2015 Canary Wharf Squash Classic'), ('objName', 'squash')]
[('sName', 'Canary Wharf Squash Classic'), ('objName', 'squash')]
[('sName', '2013 Canary Wharf Squash Classic'), ('objName', 'squash')]
[('sName', '2014 FIM Motocross World Championship season'), ('objName', 'motocross')]
[('sName', 'beach soccer in Estonia'), ('objName', 'beach soccer')]
[('sName', 'Rita Keitmann'), ('objName', 'triathlon')]
[('sName', 'Till Schramm'), ('objName', 'triathlon')]
[('sName', 'Trevor Wurtele'), ('objName', 'triathlon')]
[('sName', 'Pien Keulstra'), ('objName', 'triathlon')]
[('sName', 'Mai Shiina'), ('objName', 'karate')]
[('sName', 'Keisuke Nemoto'), ('objName', 'karate')]
[('sName', 'Katsutoshi Shiina'), ('objName', 'karate')]
[('sName', "karate at the 2006 Asian Games – men's kumite +80 kg"), ('objName', 'karate')]
[('sName', "karate at the 2006 Asian Games – men's kumite 75 kg"), ('objName', 'karate')]
[('sName', "kar

In [20]:
# try looking at all object of association football P641 that are related to CR7
queryString = """
SELECT DISTINCT ?sName ?objName
WHERE {
   # bind something
   wd:Q11571 wdt:P641 ?obj .
   # get the label
   ?obj sc:name ?objName.
}
LIMIT 150
"""

print("Results")
x=run_query(queryString)

Results
[('objName', 'association football')]
1


In [2]:
# looking at association football Q2736
queryString = """
SELECT DISTINCT ?sName ?objName
WHERE {
   # bind something
   wd:Q2736 ?p ?obj .
   # get the label
   ?obj sc:name ?objName.
}
LIMIT 150
"""

print("Results")
x=run_query(queryString)

Results
[('objName', 'FIFA')]
[('objName', 'association football goal')]
[('objName', 'Category:Association football')]
[('objName', 'football')]
[('objName', 'United Kingdom of Great Britain and Ireland')]
[('objName', 'Small Brockhaus and Efron Encyclopedic Dictionary')]
[('objName', 'Olympic sport')]
[('objName', 'team sport')]
[('objName', 'Armenian Soviet Encyclopedia')]
[('objName', 'type of sport')]
[('objName', 'Great Encyclopedia of Cyril and Methodius')]
[('objName', 'Wikipedia:Vital articles')]
[('objName', 'Brockhaus and Efron Encyclopedic Dictionary')]
[('objName', 'association football player')]
[('objName', 'soccer ball')]
[('objName', 'association football pitch')]
[('objName', 'football boots')]
[('objName', 'history of association football')]
[('objName', 'kit')]
[('objName', 'WikiProject Football')]
[('objName', 'spectator sport')]
[('objName', 'Portal:Association football')]
22


In [3]:
# looking at association football Q2736
queryString = """
SELECT DISTINCT ?obj ?objName
WHERE {
   # bind something
   wd:Q2736 ?p ?obj .
   # get the label
   ?obj sc:name ?objName.
}
LIMIT 150
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q253414'), ('objName', 'FIFA')]
[('obj', 'http://www.wikidata.org/entity/Q571251'), ('objName', 'association football goal')]
[('obj', 'http://www.wikidata.org/entity/Q5611338'), ('objName', 'Category:Association football')]
[('obj', 'http://www.wikidata.org/entity/Q1081491'), ('objName', 'football')]
[('obj', 'http://www.wikidata.org/entity/Q174193'), ('objName', 'United Kingdom of Great Britain and Ireland')]
[('obj', 'http://www.wikidata.org/entity/Q19180675'), ('objName', 'Small Brockhaus and Efron Encyclopedic Dictionary')]
[('obj', 'http://www.wikidata.org/entity/Q212434'), ('objName', 'Olympic sport')]
[('obj', 'http://www.wikidata.org/entity/Q216048'), ('objName', 'team sport')]
[('obj', 'http://www.wikidata.org/entity/Q2657718'), ('objName', 'Armenian Soviet Encyclopedia')]
[('obj', 'http://www.wikidata.org/entity/Q31629'), ('objName', 'type of sport')]
[('obj', 'http://www.wikidata.org/entity/Q4091875'), ('objName', 'Great Ency

In [4]:
#try to look at all the properties related to association football player
queryString = """
SELECT DISTINCT ?p ?pName
WHERE {
   # bind something
   wd:Q2736 ?p ?obj .
   # get the label
   ?p sc:name ?pName.
}
LIMIT 150
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P3719'), ('pName', 'regulated by')]
[('p', 'http://www.wikidata.org/prop/direct/P10'), ('pName', 'video')]
[('p', 'http://www.wikidata.org/prop/direct/P1014'), ('pName', 'Art & Architecture Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1036'), ('pName', 'Dewey Decimal Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1051'), ('pName', 'PSH ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('pName', "topic's main Wikimedia portal")]
[('p', 'http://www.wikidata.org/prop/direct/P1190'), ('pName', 'Universal Decimal Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1225'), ('pName', 'U.S. National Archives Identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1245'), ('pName', 'OmegaWiki Defined Meaning')]
[('p', 'http://www.wikidata.org/prop/direct/P1282'), ('pName', 'OpenStreetMap tag or key')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('pName', 'Gran Enciclopèdia Catalan

In [5]:
# trying to look at the obj related to association football player Q937857
queryString = """
SELECT DISTINCT ?obj ?objName
WHERE {
   # bind something
   wd:Q937857 ?p ?obj .
   # get the label
   ?obj sc:name ?objName.
}
LIMIT 150
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/P2369'), ('objName', 'Soccerway player ID')]
[('obj', 'http://www.wikidata.org/entity/P3908'), ('objName', 'Reprezentacija player ID')]
[('obj', 'http://www.wikidata.org/entity/Q18536342'), ('objName', 'competitive player')]
[('obj', 'http://www.wikidata.org/entity/P2446'), ('objName', 'Transfermarkt player ID')]
[('obj', 'http://www.wikidata.org/entity/Q7217464'), ('objName', 'Category:Association football players')]
[('obj', 'http://www.wikidata.org/entity/P2823'), ('objName', 'Royal Belgian Football Association player ID')]
[('obj', 'http://www.wikidata.org/entity/Q15944511'), ('objName', 'association football team')]
[('obj', 'http://www.wikidata.org/entity/Q2736'), ('objName', 'association football')]
[('obj', 'http://www.wikidata.org/entity/Q28640'), ('objName', 'profession')]
[('obj', 'http://www.wikidata.org/entity/Q476028'), ('objName', 'association football club')]
[('obj', 'http://www.wikidata.org/entity/Q31930761'), ('objName

In [6]:
# Finally, I have found the association football club Q476028 and here is a list of all its properties
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q476028 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1709'), ('name', 'equivalent class')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2354'), ('name', 'has list')]
[('p', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'Library of Congress authority ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2671'), ('name', 'Google Knowledge Graph ID')]
[('p', 'http://www.wikidata.org/prop/direct/P268'), ('name', 'Bibliothèque nationale de France ID')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P3417'), ('name', 'Q

Final query for this task

In [11]:
# write your final query
# trying to look at the obj related to association football player Q937857
queryString = """
SELECT DISTINCT ?obj ?objName
WHERE {
   # bind something
   wd:Q937857 ?p ?obj .
   # get the label
   ?obj sc:name ?objName.
   FILTER REGEX(?objName, "association football club") 
}
LIMIT 150
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q476028'), ('objName', 'association football club')]
1


## Task 2
Identify the BGP for association football league

In [7]:
# write your queries
# From Task 1, looking at the objects related to Cristiano Ronaldo, there's Serie A (Q15804) which I suppose to be an association football league. So, I'll start by looking at its properties.
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q15804 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1132'), ('name', 'number of participants')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P1350'), ('name', 'number of matches played/races/starts')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P154'), ('name', 'logo image')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('name', 'follows')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P2002'), ('name', 'Twitter username')]
[('p', 'http://www.wikidata.org/prop/direct/P2003'), ('name', 'Instagram username')]
[('p', 'http://www.wikidata.org/prop/direct/P2013'), ('name', 'Facebook ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2094'), ('name', 'competition class')]
[('p',

In [8]:
# Now I'll look at objs that are related to Serie A Q15804
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q15804 ?p ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q194052'), ('name', 'Serie A')]
[('obj', 'http://www.wikidata.org/entity/Q6807104'), ('name', 'Category:Serie A')]
[('obj', 'http://www.wikidata.org/entity/Q122'), ('name', 'August')]
[('obj', 'http://www.wikidata.org/entity/Q2736'), ('name', 'association football')]
[('obj', 'http://www.wikidata.org/entity/Q38'), ('name', 'Italy')]
[('obj', 'http://www.wikidata.org/entity/Q631'), ('name', 'FC Internazionale Milano')]
[('obj', 'http://www.wikidata.org/entity/Q1478437'), ('name', 'association football competition')]
[('obj', 'http://www.wikidata.org/entity/Q15991303'), ('name', 'association football league')]
[('obj', 'http://www.wikidata.org/entity/Q31930761'), ('name', "men's association football")]
[('obj', 'http://www.wikidata.org/entity/Q3270632'), ('name', 'national championship')]
[('obj', 'http://www.wikidata.org/entity/Q953425'), ('name', 'Italian association football league system')]
[('obj', 'http://www.wikidata.org/entity/Q242

Final query for this task

In [13]:
# write your final query
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q15804 ?p ?obj .
   # get the label
   ?obj sc:name ?name.
   FILTER REGEX(?name, "association football league") 
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q15991303'), ('name', 'association football league')]
[('obj', 'http://www.wikidata.org/entity/Q953425'), ('name', 'Italian association football league system')]
2


## Task 3
Return for each north american country the number of association football clubs

In [15]:
# write your queries
# I'll start by looking at all the association football clubs Q476028
queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName
WHERE {
   # bind something
   ?s ?p wd:Q476028 .
   # get the label
   ?s sc:name ?sName.
   ?p sc:name ?pName.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q5611964'), ('sName', 'Template:Infobox football club'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic')]
[('s', 'http://www.wikidata.org/entity/Q51481377'), ('sName', "women's association football club"), ('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pName', 'different from')]
[('s', 'http://www.wikidata.org/entity/Q15944511'), ('sName', 'association football team'), ('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pName', 'different from')]
[('s', 'http://www.wikidata.org/entity/Q28083137'), ('sName', 'football team'), ('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pName', 'different from')]
[('s', 'http://www.wikidata.org/entity/Q103231176'), ('sName', "men's association football club"), ('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pName', 'different from')]
[('s', 'http://www.wikidata.org/entity/Q51481377'), ('sName', "women's association football club"), ('p', 'http://ww

In [15]:
# Look at the obj related to association football clubs Q476028
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q476028 ?p ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q51481377'), ('name', "women's association football club")]
[('obj', 'http://www.wikidata.org/entity/Q15944511'), ('name', 'association football team')]
[('obj', 'http://www.wikidata.org/entity/Q2736'), ('name', 'association football')]
[('obj', 'http://www.wikidata.org/entity/Q628099'), ('name', 'association football manager')]
[('obj', 'http://www.wikidata.org/entity/Q937857'), ('name', 'association football player')]
[('obj', 'http://www.wikidata.org/entity/Q714581'), ('name', 'Nea Salamis Famagusta FC')]
[('obj', 'http://www.wikidata.org/entity/Q167270'), ('name', 'trademark')]
[('obj', 'http://www.wikidata.org/entity/Q17270000'), ('name', 'football club')]
[('obj', 'http://www.wikidata.org/entity/Q28083137'), ('name', 'football team')]
[('obj', 'http://www.wikidata.org/entity/P5424'), ('name', 'Lega Serie A team ID')]
[('obj', 'http://www.wikidata.org/entity/Q5614332'), ('name', 'Category:Association football clubs')]
[('obj', 'http

In [2]:
# Look at the properties related to association football clubs
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q476028 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1709'), ('name', 'equivalent class')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2354'), ('name', 'has list')]
[('p', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'Library of Congress authority ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2671'), ('name', 'Google Knowledge Graph ID')]
[('p', 'http://www.wikidata.org/prop/direct/P268'), ('name', 'Bibliothèque nationale de France ID')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P3417'), ('name', 'Q

In [3]:
# Look at the properties of Juventus Q1422
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q1422 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P112'), ('name', 'founded by')]
[('p', 'http://www.wikidata.org/prop/direct/P115'), ('name', 'home venue')]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league')]
[('p', 'http://www.wikidata.org/prop/direct/P127'), ('name', 'owned by')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('name', 'named after')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1454'), ('name', 'legal form')]
[('p', 'http://www.wikidata.org/prop/direct/P154'), ('name', 'logo image')]
[('p', 'http://www.wikidata.org/prop/direct/P159'), ('name', 'headquarters location')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'award received')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P1830'), ('

In [4]:
#looking at the country P17 property of Juventus
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q1422 wdt:P17 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q38'), ('name', 'Italy')]
1


In [5]:
# looking at the propery of Italy Q38
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q38 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1005'), ('name', 'Portuguese National Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1036'), ('name', 'Dewey Decimal Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1081'), ('name', 'Human Development Index')]
[('p', 'http://www.wikidata.org/prop/direct/P1082'), ('name', 'population')]
[('p', 'http://www.wikidata.org/prop/direct/P1120'), ('name', 'number of deaths')]
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('name', "topic's main Wikimedia portal")]
[('p', 'http://www.wikidata.org/prop/direct/P1198'), ('name', 'unemployment rate')]
[('p', 'http://www.wikidata.org/prop/direct/P122'), ('name', 'basic form of government')]
[('p', 'http://www.wikidata.org/prop/direct/P1225'), ('name', 'U.S. National Archives Identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1245'), ('name', 'OmegaWiki Defined Meaning')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalan

In [6]:
#looking at the continent P30 of Italy
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q38 wdt:P30 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q46'), ('name', 'Europe')]
1


In [7]:
#looking at all the continents
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   ?s wdt:P30 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q60140'), ('name', 'Indian subcontinent')]
[('obj', 'http://www.wikidata.org/entity/Q55643'), ('name', 'Oceania')]
[('obj', 'http://www.wikidata.org/entity/Q19464773'), ('name', 'Northern America and Mexico')]
[('obj', 'http://www.wikidata.org/entity/Q1239'), ('name', 'Indian Ocean')]
[('obj', 'http://www.wikidata.org/entity/Q15'), ('name', 'Africa')]
[('obj', 'http://www.wikidata.org/entity/Q18'), ('name', 'South America')]
[('obj', 'http://www.wikidata.org/entity/Q4412'), ('name', 'West Africa')]
[('obj', 'http://www.wikidata.org/entity/Q46'), ('name', 'Europe')]
[('obj', 'http://www.wikidata.org/entity/Q48'), ('name', 'Asia')]
[('obj', 'http://www.wikidata.org/entity/Q49'), ('name', 'North America')]
[('obj', 'http://www.wikidata.org/entity/Q51'), ('name', 'Antarctica')]
[('obj', 'http://www.wikidata.org/entity/Q538'), ('name', 'Insular Oceania')]
[('obj', 'http://www.wikidata.org/entity/Q5401'), ('name', 'Eurasia')]
[('obj', 'http://

In [9]:
# So I have found the BGP for North America Q49
# Now I'll try to find all the association fooaball clubs of North America

queryString = """
SELECT DISTINCT ?club ?name
WHERE {
   # bind something
   ?club wdt:P31 wd:Q476028 .
   wd:Q476028 wdt:P17 ?country .
   ?country wdt:P30 wd:Q49 .
   # get the label
   ?club sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [3]:
queryString = """
SELECT DISTINCT ?club ?name
WHERE {
   # bind something
   #?club wdt:P31 wd:Q476028 .
   wd:Q476028 wdt:P17 ?country .
   ?country wdt:P30 wd:Q49 .
   # get the label
   ?club sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [30]:
# the problem is that I was looking for the property country P17 of a association football club, but there isn't any property country for association football club.
# i will try to look at all the objs related to 
queryString = """
SELECT DISTINCT ?club ?name
WHERE {
   # bind something
   ?club wdt:P31 wd:Q476028 .
   # get the label
   ?club sc:name ?name.
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('club', 'http://www.wikidata.org/entity/Q4647107'), ('name', 'A.D. Dili Leste')]
[('club', 'http://www.wikidata.org/entity/Q8186475'), ('name', 'Tahuichi Academy')]
[('club', 'http://www.wikidata.org/entity/Q8208548'), ('name', 'Atlético Trujillo')]
[('club', 'http://www.wikidata.org/entity/Q2631783'), ('name', 'Cizrespor')]
[('club', 'http://www.wikidata.org/entity/Q3448775'), ('name', 'FK Sloboda Mrkonjić Grad')]
[('club', 'http://www.wikidata.org/entity/Q5100109'), ('name', 'Chinandega FC')]
[('club', 'http://www.wikidata.org/entity/Q5379014'), ('name', 'Enkaspor')]
[('club', 'http://www.wikidata.org/entity/Q7280157'), ('name', "Radford Highlanders men's soccer")]
[('club', 'http://www.wikidata.org/entity/Q7300846'), ('name', 'Real Arroyo Seco')]
[('club', 'http://www.wikidata.org/entity/Q14252413'), ('name', 'Rapid JC')]
[('club', 'http://www.wikidata.org/entity/Q14919844'), ('name', "Fortuna '54")]
[('club', 'http://www.wikidata.org/entity/Q3375656'), ('name', 'Persekabp

In [28]:
# try to look if Juventus is an instance of association football club, so I can better understand the problem of match countries and clubs

queryString = """
SELECT DISTINCT ?club ?name
WHERE {
   # bind something
   ?club wdt:P31 wd:Q476028 .
   
   # get the label
   ?club sc:name ?name.
   FILTER REGEX(?name, "Juventus F.C.") 
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('club', 'http://www.wikidata.org/entity/Q1422'), ('name', 'Juventus F.C.')]
[('club', 'http://www.wikidata.org/entity/Q3395169'), ('name', 'Juventus F.C. Youth Sector')]
2


In [29]:
# I have understand the problem, it was that I was looking at country of association football club instead of looking at the club retrieved from the first row of the where clause 
# So, I will look at the country property of the club, instead of the association football club.
queryString = """
SELECT DISTINCT ?club ?name
WHERE {
   # bind something
   ?club wdt:P31 wd:Q476028 .
   ?club wdt:P17 ?country .
   ?country wdt:P30 wd:Q49 .
   # get the label
   ?club sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('club', 'http://www.wikidata.org/entity/Q7280157'), ('name', "Radford Highlanders men's soccer")]
[('club', 'http://www.wikidata.org/entity/Q15876988'), ('name', 'Grenades FC')]
[('club', 'http://www.wikidata.org/entity/Q3930989'), ('name', 'Real Colorado Foxes')]
[('club', 'http://www.wikidata.org/entity/Q4816106'), ('name', 'Atlanta Magic')]
[('club', 'http://www.wikidata.org/entity/Q4852791'), ('name', 'Baltimore Bohemians')]
[('club', 'http://www.wikidata.org/entity/Q5020506'), ('name', 'California Gold')]
[('club', 'http://www.wikidata.org/entity/Q5087877'), ('name', 'Chattanooga Express')]
[('club', 'http://www.wikidata.org/entity/Q5095562'), ('name', 'Chicago Croatian')]
[('club', 'http://www.wikidata.org/entity/Q5266043'), ('name', 'Waza Flo')]
[('club', 'http://www.wikidata.org/entity/Q6116983'), ('name', 'Jackson Chargers')]
[('club', 'http://www.wikidata.org/entity/Q7013918'), ('name', 'New York Red Bulls U-23')]
[('club', 'http://www.wikidata.org/entity/Q7389397')

In [19]:
#now, I will try to count on the group by country
queryString = """
SELECT DISTINCT ?country (COUNT(?club) as ?clubs)
WHERE {
   # bind something
   ?club wdt:P31 wd:Q476028 ;
           wdt:P17 ?country .
   ?country wdt:P30 wd:Q49 .
   # get the label
   ?club sc:name ?name.
}
GROUP BY (?country)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q757'), ('clubs', '11')]
[('country', 'http://www.wikidata.org/entity/Q804'), ('clubs', '32')]
[('country', 'http://www.wikidata.org/entity/Q778'), ('clubs', '9')]
[('country', 'http://www.wikidata.org/entity/Q786'), ('clubs', '13')]
[('country', 'http://www.wikidata.org/entity/Q754'), ('clubs', '22')]
[('country', 'http://www.wikidata.org/entity/Q29999'), ('clubs', '3')]
[('country', 'http://www.wikidata.org/entity/Q769'), ('clubs', '1')]
[('country', 'http://www.wikidata.org/entity/Q30'), ('clubs', '1193')]
[('country', 'http://www.wikidata.org/entity/Q783'), ('clubs', '90')]
[('country', 'http://www.wikidata.org/entity/Q242'), ('clubs', '35')]
[('country', 'http://www.wikidata.org/entity/Q800'), ('clubs', '45')]
[('country', 'http://www.wikidata.org/entity/Q1183'), ('clubs', '1')]
[('country', 'http://www.wikidata.org/entity/Q16'), ('clubs', '136')]
[('country', 'http://www.wikidata.org/entity/Q223'), ('clubs', '12')]
[('country',

In [15]:
#now, I will try to count on the group by country
queryString = """
SELECT DISTINCT ?name
WHERE {
   # get the label
   wd:Q774 sc:name ?name.
}
GROUP BY (?country)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('name', 'Guatemala')]
1


In [24]:
#now, I will try to count on the group by country
queryString = """
SELECT DISTINCT ?name (COUNT(?club) as ?clubs)
WHERE {
   # bind something
   ?club wdt:P31 wd:Q476028 ;
           wdt:P17 ?country .
   ?country wdt:P30 wd:Q49 .
   # get the label
   ?country sc:name ?name.
}
GROUP BY (?name)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('name', 'Jamaica'), ('clubs', '30')]
[('name', 'Cuba'), ('clubs', '21')]
[('name', 'Mexico'), ('clubs', '355')]
[('name', 'Greenland'), ('clubs', '12')]
[('name', 'Panama'), ('clubs', '36')]
[('name', 'Barbados'), ('clubs', '10')]
[('name', 'Russian Empire'), ('clubs', '3')]
[('name', 'Saint Vincent and the Grenadines'), ('clubs', '11')]
[('name', 'Guatemala'), ('clubs', '45')]
[('name', 'Costa Rica'), ('clubs', '91')]
[('name', 'Dominican Republic'), ('clubs', '13')]
[('name', 'El Salvador'), ('clubs', '104')]
[('name', 'Saint Kitts and Nevis'), ('clubs', '10')]
[('name', 'United States of America'), ('clubs', '1199')]
[('name', 'Denmark'), ('clubs', '267')]
[('name', 'Haiti'), ('clubs', '66')]
[('name', 'Turks and Caicos Islands'), ('clubs', '12')]
[('name', 'Canada'), ('clubs', '138')]
[('name', 'Belize'), ('clubs', '100')]
[('name', 'The Bahamas'), ('clubs', '45')]
[('name', 'Honduras'), ('clubs', '93')]
[('name', 'Dominica'), ('clubs', '54')]
[('name', 'Trinidad and Toba

Final query for this task

In [5]:
# write your final query
queryString = """
SELECT DISTINCT ?country ?countryName (COUNT(?club) as ?numOfClubs)
WHERE {
   # bind something
   ?club wdt:P31 wd:Q476028 ;
           wdt:P17 ?country .
   ?country wdt:P30 wd:Q49 .
   # get the label
   ?country sc:name ?countryName.
}
GROUP BY ?country ?countryName
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q754'), ('countryName', 'Trinidad and Tobago'), ('numOfClubs', '22')]
[('country', 'http://www.wikidata.org/entity/Q16'), ('countryName', 'Canada'), ('numOfClubs', '138')]
[('country', 'http://www.wikidata.org/entity/Q778'), ('countryName', 'The Bahamas'), ('numOfClubs', '45')]
[('country', 'http://www.wikidata.org/entity/Q11703'), ('countryName', 'United States Virgin Islands'), ('numOfClubs', '1')]
[('country', 'http://www.wikidata.org/entity/Q786'), ('countryName', 'Dominican Republic'), ('numOfClubs', '13')]
[('country', 'http://www.wikidata.org/entity/Q763'), ('countryName', 'Saint Kitts and Nevis'), ('numOfClubs', '10')]
[('country', 'http://www.wikidata.org/entity/Q766'), ('countryName', 'Jamaica'), ('numOfClubs', '30')]
[('country', 'http://www.wikidata.org/entity/Q790'), ('countryName', 'Haiti'), ('numOfClubs', '66')]
[('country', 'http://www.wikidata.org/entity/Q1183'), ('countryName', 'Puerto Rico'), ('numOfClubs', '1')]
[

## Task 4
Identify the BGP for the UEFA Champions League.

In [31]:
# write your queries
# try to look at the properties association football league Q15991303
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q15991303 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1709'), ('name', 'equivalent class')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type')]
[('p', 'http://www.wikidata.org/prop/direct/P2671'), ('name', 'Google Knowledge Graph ID')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport')]
[('p', 'http://www.wikidata.org/prop/direct/P8408'), ('name', 'KBpedia ID')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('name', "topic's main category")]
9


In [32]:
# try to look at the objs related to association football league Q15991303
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q15991303 ?p ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/P2499'), ('name', 'league level above')]
[('obj', 'http://www.wikidata.org/entity/P2500'), ('name', 'league level below')]
[('obj', 'http://www.wikidata.org/entity/P3983'), ('name', 'sports league level')]
[('obj', 'http://www.wikidata.org/entity/Q26887310'), ('name', 'association football team season')]
[('obj', 'http://www.wikidata.org/entity/P17'), ('name', 'country')]
[('obj', 'http://www.wikidata.org/entity/Q2736'), ('name', 'association football')]
[('obj', 'http://www.wikidata.org/entity/Q1478437'), ('name', 'association football competition')]
[('obj', 'http://www.wikidata.org/entity/P641'), ('name', 'sport')]
[('obj', 'http://www.wikidata.org/entity/Q623109'), ('name', 'sports league')]
[('obj', 'http://www.wikidata.org/entity/P664'), ('name', 'organizer')]
[('obj', 'http://www.wikidata.org/entity/P1132'), ('name', 'number of participants')]
[('obj', 'http://www.wikidata.org/entity/Q6383739'), ('name', 'Category:Association foot

In [ ]:
# looking at 200 association football leagues
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   # bind something
   ?s ?p wd:Q15991303 .
   # get the label
   ?s sc:name ?name.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q8175816'), ('name', 'Category:1994 in Swedish association football leagues')]
[('s', 'http://www.wikidata.org/entity/Q8182829'), ('name', 'Category:2000 in Swedish association football leagues')]
[('s', 'http://www.wikidata.org/entity/Q304228'), ('name', 'Uzbekistan Pro League')]
[('s', 'http://www.wikidata.org/entity/Q2950655'), ('name', 'Ligue 2')]
[('s', 'http://www.wikidata.org/entity/Q2951924'), ('name', "Moldovan women's football championship")]
[('s', 'http://www.wikidata.org/entity/Q973312'), ('name', 'Championnat National 3')]
[('s', 'http://www.wikidata.org/entity/Q2792041'), ('name', '3. divisjon')]
[('s', 'http://www.wikidata.org/entity/Q1162407'), ('name', 'Nemzeti Bajnokság III')]
[('s', 'http://www.wikidata.org/entity/Q1476473'), ('name', 'Landesliga Niederrhein')]
[('s', 'http://www.wikidata.org/entity/Q162533'), ('name', 'Russian Football National League Competition')]
[('s', 'http://www.wikidata.org/entity/Q1707697'), ('

In [39]:
# filter on association football leagues
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   # bind something
   ?s ?p wd:Q15991303 .
   # get the label
   ?s sc:name ?name.
   FILTER REGEX (?name, "[*]Champions League[*]")
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [ ]:
# since the previous regex was wrong, filter on association football leagues 
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   # bind something
   ?s ?p wd:Q15991303 .
   # get the label
   ?s sc:name ?name.
   FILTER REGEX (?name, ".*Champions League")
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q17632439'), ('name', 'VFF Champions League')]
1


In [51]:
# try to look at the properties of the subjects related to the association football leagues
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   ?s ?p wd:Q15991303 .
   # get the label
   ?p sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1423'), ('name', 'template has topic')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('name', 'named after')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P301'), ('name', "category's main topic")]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P3450'), ('name', 'sports season of league or competition')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport')]
[('p', 'http://www.wikidata.org/prop/direct/P971'), ('name', 'category combines topics')]
9


In [54]:
# try to look at the objs related to sports season of league or competition P3450
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   ?s wdt:P3450 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q29762'), ('name', 'fencing at the Summer Olympics')]
[('obj', 'http://www.wikidata.org/entity/Q3066961'), ('name', 'Galician trainera league')]
[('obj', 'http://www.wikidata.org/entity/Q610674'), ('name', 'soccer in the United States')]
[('obj', 'http://www.wikidata.org/entity/Q7854'), ('name', 'Argentine Grand Prix')]
[('obj', 'http://www.wikidata.org/entity/Q1070262'), ('name', 'European Touring Car Cup')]
[('obj', 'http://www.wikidata.org/entity/Q1774675'), ('name', 'División de Honor de Waterpolo')]
[('obj', 'http://www.wikidata.org/entity/Q2023337'), ('name', 'División de Honor Juvenil de Fútbol')]
[('obj', 'http://www.wikidata.org/entity/Q204348'), ('name', 'Continental Basketball Association')]
[('obj', 'http://www.wikidata.org/entity/Q2744659'), ('name', 'Copa de la Reina de Baloncesto')]
[('obj', 'http://www.wikidata.org/entity/Q303871'), ('name', 'European Rally Championship')]
[('obj', 'http://www.wikidata.org/entity/Q368165'

In [55]:
# try to filter on the champions league
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   ?s wdt:P3450 ?obj .
   # get the label
   ?obj sc:name ?name.
   FILTER REGEX (?name, ".*Champions League.*")
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q652498'), ('name', "EHF Women's Champions League")]
[('obj', 'http://www.wikidata.org/entity/Q648474'), ('name', 'OFC Champions League')]
[('obj', 'http://www.wikidata.org/entity/Q300098'), ('name', 'CEV Champions League')]
[('obj', 'http://www.wikidata.org/entity/Q13416183'), ('name', '2015–16 UEFA Champions League')]
[('obj', 'http://www.wikidata.org/entity/Q219261'), ('name', 'CAF Champions League')]
[('obj', 'http://www.wikidata.org/entity/Q3240479'), ('name', 'African Handball Champions League')]
[('obj', 'http://www.wikidata.org/entity/Q18740961'), ('name', '2016–17 UEFA Champions League')]
[('obj', 'http://www.wikidata.org/entity/Q20972117'), ('name', '2017–18 UEFA Champions League')]
[('obj', 'http://www.wikidata.org/entity/Q941782'), ('name', 'LEN Champions League')]
[('obj', 'http://www.wikidata.org/entity/Q22340390'), ('name', 'Basketball Champions League')]
[('obj', 'http://www.wikidata.org/entity/Q18756'), ('name', 'UEFA Ch

Final query for this task

In [53]:
# write your final query
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   ?s wdt:P3450 ?obj .
   # get the label
   ?obj sc:name ?name.
   FILTER REGEX (?name, "UEFA Champions League")
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q13416183'), ('name', '2015–16 UEFA Champions League')]
[('obj', 'http://www.wikidata.org/entity/Q18740961'), ('name', '2016–17 UEFA Champions League')]
[('obj', 'http://www.wikidata.org/entity/Q20972117'), ('name', '2017–18 UEFA Champions League')]
[('obj', 'http://www.wikidata.org/entity/Q18756'), ('name', 'UEFA Champions League')]
4


## Task 5
Consider the countries which have at least two different clubs of their nationality who won the UEFA Champions League. Return the list of these countries with the number of different clubs who won the UEFA Champions League (the result set must be triples of country IRI, label and #clubs).

In [3]:
# write your queries
# I try to look at all the Champions League winners
# start by looking at the properties of Champions League Q18756
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q18756 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1132'), ('name', 'number of participants')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P154'), ('name', 'logo image')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P2002'), ('name', 'Twitter username')]
[('p', 'http://www.wikidata.org/prop/direct/P2003'), ('name', 'Instagram username')]
[('p', 'http://www.wikidata.org/prop/direct/P2013'), ('name', 'Facebook ID')]
[('p', 'http://www.wikidata.org/prop/direct/P214'), ('name', 'VIAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2257'), ('name', 'event interval')]
[('p', 'http:/

In [4]:
# try to look at the winner of the champions league
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q18756 wdt:P1346 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q631'), ('name', 'FC Internazionale Milano')]
1


In [5]:
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   ?s ?p wd:Q18756 .
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1423'), ('name', 'template has topic')]
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('name', 'facet of')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')]
[('p', 'http://www.wikidata.org/prop/direct/P301'), ('name', "category's main topic")]
[('p', 'http://www.wikidata.org/prop/direct/P3450'), ('name', 'sports season of league or competition')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('name', 'part of')]
[('p', 'http://www.wikidata.org/prop/direct/P921'), ('name', 'main subject')]
[('p', 'http://www.wikidata.org/prop/direct/P971'), ('name', 'category combines topics')]
[('p', 'http://www.wikidata.org/prop/direct/P2499'), ('name', 'league level above')]
[('p', 'http://www.wikidata.org/prop/direct/P3085'), ('name', 'qualifies for event')]
10


In [6]:
# try to look at the properties of the 2017–18 UEFA Champions League
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q20972117 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1132'), ('name', 'number of participants')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P1350'), ('name', 'number of matches played/races/starts')]
[('p', 'http://www.wikidata.org/prop/direct/P2348'), ('name', 'time period')]
[('p', 'http://www.wikidata.org/prop/direct/P2671'), ('name', 'Google Knowledge Graph ID')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P3450'), ('name', 'sports season of league or competition')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('name', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P393'), ('name', 'edition number')]
[('p', 'http://www.wikidata.org/prop/direct/P580'), ('name', 'start time')]
[('p', 'http://www.wikidata.org/prop/direct/P582'), ('name', 'end time')]
[('p', 'http://www.wikidata.org/prop/direct/P585'), ('name', 'p

In [7]:
# try to look at the winner of the 2017–18 UEFA Champions League
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q20972117 wdt:P1346 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q8682'), ('name', 'Real Madrid CF')]
1


In [8]:
#try to look at the instance of the 2017–18 UEFA Champions League
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q20972117 wdt:P31 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q27020041'), ('name', 'sports season')]
1


In [9]:
#try to look at the instance of the 2017–18 UEFA Champions League
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q20972117 wdt:P3450 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q18756'), ('name', 'UEFA Champions League')]
1


In [10]:
#try to get all the champions league Q18756 competitions
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   # bind something
   ?s wdt:P3450 wd:Q18756 .
   # get the label
   ?s sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q497052'), ('name', '1964–65 European Cup')]
[('s', 'http://www.wikidata.org/entity/Q500084'), ('name', '1963–64 European Cup')]
[('s', 'http://www.wikidata.org/entity/Q30608'), ('name', '2013–14 UEFA Champions League')]
[('s', 'http://www.wikidata.org/entity/Q13416183'), ('name', '2015–16 UEFA Champions League')]
[('s', 'http://www.wikidata.org/entity/Q269535'), ('name', '1991–92 European Cup')]
[('s', 'http://www.wikidata.org/entity/Q308996'), ('name', '2004–05 UEFA Champions League')]
[('s', 'http://www.wikidata.org/entity/Q696237'), ('name', '1988–89 European Cup')]
[('s', 'http://www.wikidata.org/entity/Q696245'), ('name', '1986–87 European Cup')]
[('s', 'http://www.wikidata.org/entity/Q219384'), ('name', '1955–56 European Cup')]
[('s', 'http://www.wikidata.org/entity/Q456097'), ('name', '2000–01 UEFA Champions League')]
[('s', 'http://www.wikidata.org/entity/Q18740961'), ('name', '2016–17 UEFA Champions League')]
[('s', 'http://www.w

In [12]:
# order the list by name
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   # bind something
   ?s wdt:P3450 wd:Q18756 .
   # get the label
   ?s sc:name ?name.
}
ORDER BY ?name
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q219384'), ('name', '1955–56 European Cup')]
[('s', 'http://www.wikidata.org/entity/Q245202'), ('name', '1956–57 European Cup')]
[('s', 'http://www.wikidata.org/entity/Q309318'), ('name', '1957–58 European Cup')]
[('s', 'http://www.wikidata.org/entity/Q309316'), ('name', '1958–59 European Cup')]
[('s', 'http://www.wikidata.org/entity/Q392535'), ('name', '1959–60 European Cup')]
[('s', 'http://www.wikidata.org/entity/Q330686'), ('name', '1960–61 European Cup')]
[('s', 'http://www.wikidata.org/entity/Q497028'), ('name', '1961–62 European Cup')]
[('s', 'http://www.wikidata.org/entity/Q500074'), ('name', '1962–63 European Cup')]
[('s', 'http://www.wikidata.org/entity/Q500084'), ('name', '1963–64 European Cup')]
[('s', 'http://www.wikidata.org/entity/Q497052'), ('name', '1964–65 European Cup')]
[('s', 'http://www.wikidata.org/entity/Q573225'), ('name', '1965–66 European Cup')]
[('s', 'http://www.wikidata.org/entity/Q586977'), ('name', '1966–67 

In [14]:
# for each of them, take the winner

queryString = """
SELECT DISTINCT ?s ?name ?winner ?winnerName
WHERE {
   # bind something
   ?s wdt:P3450 wd:Q18756 ;
           wdt:P1346 ?winner .
   # get the label
   ?s sc:name ?name.
   ?winner sc:name ?winnerName
}
ORDER BY ?name
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q219384'), ('name', '1955–56 European Cup'), ('winner', 'http://www.wikidata.org/entity/Q8682'), ('winnerName', 'Real Madrid CF')]
[('s', 'http://www.wikidata.org/entity/Q245202'), ('name', '1956–57 European Cup'), ('winner', 'http://www.wikidata.org/entity/Q8682'), ('winnerName', 'Real Madrid CF')]
[('s', 'http://www.wikidata.org/entity/Q309318'), ('name', '1957–58 European Cup'), ('winner', 'http://www.wikidata.org/entity/Q8682'), ('winnerName', 'Real Madrid CF')]
[('s', 'http://www.wikidata.org/entity/Q309316'), ('name', '1958–59 European Cup'), ('winner', 'http://www.wikidata.org/entity/Q8682'), ('winnerName', 'Real Madrid CF')]
[('s', 'http://www.wikidata.org/entity/Q392535'), ('name', '1959–60 European Cup'), ('winner', 'http://www.wikidata.org/entity/Q8682'), ('winnerName', 'Real Madrid CF')]
[('s', 'http://www.wikidata.org/entity/Q330686'), ('name', '1960–61 European Cup'), ('winner', 'http://www.wikidata.org/entity/Q131499'), ('wi

In [35]:
# debugging errors
queryString = """
SELECT DISTINCT ?winnerName
WHERE {
   # bind something
   wd:Q189180 wdt:P1346 ?winner .
   # get the label
   ?winner sc:name ?winnerName .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('winnerName', 'Manchester United F.C.')]
1


Final query for this task

In [36]:
# debugging errors
queryString = """
SELECT DISTINCT ?winnerName ?countryName
WHERE {
   # bind something
   wd:Q189180 wdt:P1346 ?winner .
   ?winner wdt:P17 ?country .
   # get the label
   ?winner sc:name ?winnerName .
   ?country sc:name ?countryName .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('winnerName', 'Manchester United F.C.'), ('countryName', 'United Kingdom')]
1


In [39]:
# for each winner, take its country
queryString = """
SELECT DISTINCT ?UCLEditionName ?winnerName ?countryName
WHERE {
   # bind something
   ?UCLEdition wdt:P3450 wd:Q18756 ;
           wdt:P1346 ?winner .
   ?winner wdt:P17 ?country .
   # get the label
   ?UCLEdition sc:name ?UCLEditionName .
   ?winner sc:name ?winnerName .
   ?country sc:name ?countryName .
}
ORDER BY ?UCLEditionName
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('UCLEditionName', '1955–56 European Cup'), ('winnerName', 'Real Madrid CF'), ('countryName', 'Spain')]
[('UCLEditionName', '1956–57 European Cup'), ('winnerName', 'Real Madrid CF'), ('countryName', 'Spain')]
[('UCLEditionName', '1957–58 European Cup'), ('winnerName', 'Real Madrid CF'), ('countryName', 'Spain')]
[('UCLEditionName', '1958–59 European Cup'), ('winnerName', 'Real Madrid CF'), ('countryName', 'Spain')]
[('UCLEditionName', '1959–60 European Cup'), ('winnerName', 'Real Madrid CF'), ('countryName', 'Spain')]
[('UCLEditionName', '1960–61 European Cup'), ('winnerName', 'S.L. Benfica'), ('countryName', 'Portugal')]
[('UCLEditionName', '1961–62 European Cup'), ('winnerName', 'S.L. Benfica'), ('countryName', 'Portugal')]
[('UCLEditionName', '1962–63 European Cup'), ('winnerName', 'A.C. Milan'), ('countryName', 'Italy')]
[('UCLEditionName', '1963–64 European Cup'), ('winnerName', 'FC Internazionale Milano'), ('countryName', 'Italy')]
[('UCLEditionName', '1964–65 European C

In [41]:
# now group by country
queryString = """
SELECT DISTINCT ?country ?countryName
WHERE {
   # bind something
   ?UCLEdition wdt:P3450 wd:Q18756 ;
           wdt:P1346 ?winner .
   ?winner wdt:P17 ?country .
   # get the label
   ?UCLEdition sc:name ?UCLEditionName .
   ?winner sc:name ?winnerName .
   ?country sc:name ?countryName .
}
GROUP BY (?country)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q218'), ('countryName', 'Romania')]
[('country', 'http://www.wikidata.org/entity/Q38'), ('countryName', 'Italy')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('countryName', 'Germany')]
[('country', 'http://www.wikidata.org/entity/Q403'), ('countryName', 'Serbia')]
[('country', 'http://www.wikidata.org/entity/Q29'), ('countryName', 'Spain')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France')]
[('country', 'http://www.wikidata.org/entity/Q29999'), ('countryName', 'Kingdom of the Netherlands')]
[('country', 'http://www.wikidata.org/entity/Q45'), ('countryName', 'Portugal')]
[('country', 'http://www.wikidata.org/entity/Q55'), ('countryName', 'Netherlands')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('countryName', 'United Kingdom')]
10


In [63]:
queryString = """

SELECT DISTINCT ?country (COUNT(?winner) AS ?winners)
WHERE {
    # bind something
    ?UCLEdition wdt:P3450 wd:Q18756 ;
            wdt:P1346 ?winner .
    ?winner wdt:P17 ?country .
    # get the label
    ?country sc:name ?countryName .
    ?winner sc:name ?winnerName .
}
GROUP BY ?country
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q38'), ('winners', '12')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('winners', '8')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('winners', '16')]
[('country', 'http://www.wikidata.org/entity/Q29999'), ('winners', '1')]
[('country', 'http://www.wikidata.org/entity/Q218'), ('winners', '1')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('winners', '1')]
[('country', 'http://www.wikidata.org/entity/Q55'), ('winners', '6')]
[('country', 'http://www.wikidata.org/entity/Q45'), ('winners', '4')]
[('country', 'http://www.wikidata.org/entity/Q29'), ('winners', '17')]
[('country', 'http://www.wikidata.org/entity/Q403'), ('winners', '1')]
10


In [77]:
queryString = """

SELECT DISTINCT ?country ?countryName (COUNT(?winner) AS ?winners)
WHERE {
    # bind something
    ?UCLEdition wdt:P3450 wd:Q18756 ;
            wdt:P1346 ?winner .
    ?winner wdt:P17 ?country .
    # get the label
    ?country sc:name ?countryName .
    ?winner sc:name ?winnerName .
}
GROUP BY ?country ?countryName
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q218'), ('countryName', 'Romania'), ('winners', '1')]
[('country', 'http://www.wikidata.org/entity/Q38'), ('countryName', 'Italy'), ('winners', '12')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('countryName', 'Germany'), ('winners', '8')]
[('country', 'http://www.wikidata.org/entity/Q403'), ('countryName', 'Serbia'), ('winners', '1')]
[('country', 'http://www.wikidata.org/entity/Q29'), ('countryName', 'Spain'), ('winners', '17')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France'), ('winners', '1')]
[('country', 'http://www.wikidata.org/entity/Q29999'), ('countryName', 'Kingdom of the Netherlands'), ('winners', '1')]
[('country', 'http://www.wikidata.org/entity/Q45'), ('countryName', 'Portugal'), ('winners', '4')]
[('country', 'http://www.wikidata.org/entity/Q55'), ('countryName', 'Netherlands'), ('winners', '6')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('countryName', 'United Kingdo

In [83]:
#this retrieve all the UCL wins for a country
queryString = """

SELECT DISTINCT ?country ?countryName (COUNT(?winner) AS ?numOfWinners)
WHERE {
    # bind something
    ?UCLEdition wdt:P3450 wd:Q18756 ;
            wdt:P1346 ?winner .
    ?winner wdt:P17 ?country .
    # get the label
    ?country sc:name ?countryName .
    ?winner sc:name ?winnerName .
}
GROUP BY ?country ?countryName
HAVING (COUNT(?winner) > 1)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q38'), ('countryName', 'Italy'), ('numOfWinners', '12')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('countryName', 'Germany'), ('numOfWinners', '8')]
[('country', 'http://www.wikidata.org/entity/Q29'), ('countryName', 'Spain'), ('numOfWinners', '17')]
[('country', 'http://www.wikidata.org/entity/Q45'), ('countryName', 'Portugal'), ('numOfWinners', '4')]
[('country', 'http://www.wikidata.org/entity/Q55'), ('countryName', 'Netherlands'), ('numOfWinners', '6')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('countryName', 'United Kingdom'), ('numOfWinners', '16')]
6


In [84]:
#select all the distinct winners of UCL
queryString = """

SELECT DISTINCT ?winner ?winnerName
WHERE {
    # bind something
    ?UCLEdition wdt:P3450 wd:Q18756 ;
            wdt:P1346 ?winner .
    
    # get the label
    ?winner sc:name ?winnerName .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('winner', 'http://www.wikidata.org/entity/Q179658'), ('winnerName', 'FCSB')]
[('winner', 'http://www.wikidata.org/entity/Q1130849'), ('winnerName', 'Liverpool F.C.')]
[('winner', 'http://www.wikidata.org/entity/Q132885'), ('winnerName', 'Olympique de Marseille')]
[('winner', 'http://www.wikidata.org/entity/Q1422'), ('winnerName', 'Juventus F.C.')]
[('winner', 'http://www.wikidata.org/entity/Q1543'), ('winnerName', 'A.C. Milan')]
[('winner', 'http://www.wikidata.org/entity/Q18656'), ('winnerName', 'Manchester United F.C.')]
[('winner', 'http://www.wikidata.org/entity/Q18711'), ('winnerName', 'Aston Villa F.C.')]
[('winner', 'http://www.wikidata.org/entity/Q631'), ('winnerName', 'FC Internazionale Milano')]
[('winner', 'http://www.wikidata.org/entity/Q7156'), ('winnerName', 'FC Barcelona')]
[('winner', 'http://www.wikidata.org/entity/Q11938'), ('winnerName', 'PSV Eindhoven')]
[('winner', 'http://www.wikidata.org/entity/Q15789'), ('winnerName', 'FC Bayern Munich')]
[('winner', '

In [96]:
#this retrieve all the countries that has at least 2 club winners of the UCL
queryString = """

SELECT DISTINCT ?country ?countryName (COUNT(?winner) AS ?numOfWinners)
WHERE {
    
    ?winner wdt:P17 ?country .
    # get the label
    ?country sc:name ?countryName .
    {
    SELECT DISTINCT ?winner
    WHERE {
            # bind something
            ?UCLEdition wdt:P3450 wd:Q18756 ;
                wdt:P1346 ?winner .
    }
    }
}
GROUP BY ?country ?countryName
HAVING (COUNT(?winner) > 1)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q38'), ('countryName', 'Italy'), ('numOfWinners', '3')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('countryName', 'Germany'), ('numOfWinners', '3')]
[('country', 'http://www.wikidata.org/entity/Q29'), ('countryName', 'Spain'), ('numOfWinners', '2')]
[('country', 'http://www.wikidata.org/entity/Q45'), ('countryName', 'Portugal'), ('numOfWinners', '2')]
[('country', 'http://www.wikidata.org/entity/Q55'), ('countryName', 'Netherlands'), ('numOfWinners', '3')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('countryName', 'United Kingdom'), ('numOfWinners', '6')]
6


In [97]:
# write your final query
queryString = """

SELECT DISTINCT ?country ?countryName (COUNT(?winner) AS ?numOfWinners)
WHERE {
    
    ?winner wdt:P17 ?country .
    # get the label
    ?country sc:name ?countryName .
    {
    SELECT DISTINCT ?winner
    WHERE {
            # bind something
            ?UCLEdition wdt:P3450 wd:Q18756 ;
                wdt:P1346 ?winner .
    }
    }
}
GROUP BY ?country ?countryName
HAVING (COUNT(?winner) > 1)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q38'), ('countryName', 'Italy'), ('numOfWinners', '3')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('countryName', 'Germany'), ('numOfWinners', '3')]
[('country', 'http://www.wikidata.org/entity/Q29'), ('countryName', 'Spain'), ('numOfWinners', '2')]
[('country', 'http://www.wikidata.org/entity/Q45'), ('countryName', 'Portugal'), ('numOfWinners', '2')]
[('country', 'http://www.wikidata.org/entity/Q55'), ('countryName', 'Netherlands'), ('numOfWinners', '3')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('countryName', 'United Kingdom'), ('numOfWinners', '6')]
6


## Task 6
Return for each european country the top level league (the result set must be a list of country IRI and label, and league IRI and label).

In [6]:
# write your queries
# I will start by looking at the Q15804 Serie A's properties and objs
queryString = """
SELECT DISTINCT ?p ?pname ?obj ?objname
WHERE {
   # bind something
   wd:Q15804 ?p ?obj .
   # get the label
   ?p sc:name ?pname.
   ?obj sc:name ?objname.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2500'), ('pname', 'league level below'), ('obj', 'http://www.wikidata.org/entity/Q194052'), ('objname', 'Serie A')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pname', "topic's main category"), ('obj', 'http://www.wikidata.org/entity/Q6807104'), ('objname', 'Category:Serie A')]
[('p', 'http://www.wikidata.org/prop/direct/P4794'), ('pname', 'season starts'), ('obj', 'http://www.wikidata.org/entity/Q122'), ('objname', 'August')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pname', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q2736'), ('objname', 'association football')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country'), ('obj', 'http://www.wikidata.org/entity/Q38'), ('objname', 'Italy')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pname', 'winner'), ('obj', 'http://www.wikidata.org/entity/Q631'), ('objname', 'FC Internazionale Milano')]
[('p', 'http://www.wikidata.org/prop/direct

In [8]:
# I will start by looking at the Q15804 Serie A's properties and objs
queryString = """
SELECT DISTINCT ?s ?sname ?p ?pname
WHERE {
   # bind something
   ?s ?p wd:Q15804 .
   # get the label
   ?p sc:name ?pname.
   ?s sc:name ?sname.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q45901'), ('sname', 'Thierry Henry'), ('p', 'http://www.wikidata.org/prop/direct/P118'), ('pname', 'league')]
[('s', 'http://www.wikidata.org/entity/Q223724'), ('sname', '2009–10 Serie A'), ('p', 'http://www.wikidata.org/prop/direct/P3450'), ('pname', 'sports season of league or competition')]
[('s', 'http://www.wikidata.org/entity/Q240669'), ('sname', '2011–12 Serie A'), ('p', 'http://www.wikidata.org/prop/direct/P3450'), ('pname', 'sports season of league or competition')]
[('s', 'http://www.wikidata.org/entity/Q276204'), ('sname', '2010–11 Serie A'), ('p', 'http://www.wikidata.org/prop/direct/P3450'), ('pname', 'sports season of league or competition')]
[('s', 'http://www.wikidata.org/entity/Q381013'), ('sname', '2007–08 Serie A'), ('p', 'http://www.wikidata.org/prop/direct/P3450'), ('pname', 'sports season of league or competition')]
[('s', 'http://www.wikidata.org/entity/Q643978'), ('sname', '2005–06 Serie A'), ('p', 'http://www.wikid

In [11]:
# I will start by looking at the Q15804 Serie A's properties and objs
queryString = """
SELECT DISTINCT ?s ?sname ?p ?pname
WHERE {
   # bind something
   ?s ?p wd:Q15804 .
   # get the label
   ?p sc:name ?pname.
   ?s sc:name ?sname.
   FILTER REGEX(?pname, 'top')
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q6807104'), ('sname', 'Category:Serie A'), ('p', 'http://www.wikidata.org/prop/direct/P301'), ('pname', "category's main topic")]
[('s', 'http://www.wikidata.org/entity/Q7195347'), ('sname', 'Category:Serie A seasons'), ('p', 'http://www.wikidata.org/prop/direct/P971'), ('pname', 'category combines topics')]
[('s', 'http://www.wikidata.org/entity/Q10956030'), ('sname', 'Template:Serie A seasons'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pname', 'template has topic')]
[('s', 'http://www.wikidata.org/entity/Q7119823'), ('sname', 'Category:Serie A players'), ('p', 'http://www.wikidata.org/prop/direct/P971'), ('pname', 'category combines topics')]
4


In [13]:
# I will start by looking at the Q15804 Serie A's properties and objs
queryString = """
SELECT DISTINCT ?p ?pname ?obj ?objname
WHERE {
   # bind something
   wd:Q15804 ?p ?obj .
   # get the label
   ?p sc:name ?pname.
   ?obj sc:name ?objname.
   FILTER REGEX(?objname, 'top')
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [14]:
# I will start by looking at the Q15804 Serie A's properties and objs
queryString = """
SELECT DISTINCT ?p ?pname ?obj ?objname
WHERE {
   # bind something
   wd:Q15804 ?p ?obj .
   # get the label
   ?p sc:name ?pname.
   ?obj sc:name ?objname.
   FILTER REGEX(?pname, 'top')
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pname', "topic's main category"), ('obj', 'http://www.wikidata.org/entity/Q6807104'), ('objname', 'Category:Serie A')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pname', "topic's main template"), ('obj', 'http://www.wikidata.org/entity/Q10956030'), ('objname', 'Template:Serie A seasons')]
2


In [15]:
# I will try to look at Italian association football league system Q953425
queryString = """
SELECT DISTINCT ?p ?pname ?obj ?objname
WHERE {
   # bind something
   wd:Q953425 ?p ?obj .
   # get the label
   ?p sc:name ?pname.
   ?obj sc:name ?objname.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('pname', 'facet of'), ('obj', 'http://www.wikidata.org/entity/Q2736'), ('objname', 'association football')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pname', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q2736'), ('objname', 'association football')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country'), ('obj', 'http://www.wikidata.org/entity/Q38'), ('objname', 'Italy')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q1756477'), ('objname', 'league system')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q29791211'), ('objname', 'sport in a geographic region')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pname', "topic's main category"), ('obj', 'http://www.wikidata.org/entity/Q8557731'), ('objname', 'Category:Italian Football Championship')]
6

In [16]:
queryString = """
SELECT DISTINCT ?s ?sname ?p ?pname
WHERE {
   # bind something
   ?s ?p wd:Q953425 .
   # get the label
   ?p sc:name ?pname.
   ?s sc:name ?sname.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q645148'), ('sname', 'Eccellenza'), ('p', 'http://www.wikidata.org/prop/direct/P6587'), ('pname', 'league system')]
[('s', 'http://www.wikidata.org/entity/Q194052'), ('sname', 'Serie A'), ('p', 'http://www.wikidata.org/prop/direct/P6587'), ('pname', 'league system')]
[('s', 'http://www.wikidata.org/entity/Q15804'), ('sname', 'Serie A'), ('p', 'http://www.wikidata.org/prop/direct/P6587'), ('pname', 'league system')]
[('s', 'http://www.wikidata.org/entity/Q607965'), ('sname', 'Serie C'), ('p', 'http://www.wikidata.org/prop/direct/P6587'), ('pname', 'league system')]
[('s', 'http://www.wikidata.org/entity/Q677397'), ('sname', 'Lega Pro Prima Divisione'), ('p', 'http://www.wikidata.org/prop/direct/P6587'), ('pname', 'league system')]
[('s', 'http://www.wikidata.org/entity/Q1141778'), ('sname', 'Serie D'), ('p', 'http://www.wikidata.org/prop/direct/P6587'), ('pname', 'league system')]
[('s', 'http://www.wikidata.org/entity/Q8557731'), ('sname',

In [4]:
# nothing... I'll try with [('obj', 'http://www.wikidata.org/entity/Q3270632'), ('name', 'national championship')]

#[('obj', 'http://www.wikidata.org/entity/Q6807104'), ('name', 'Category:Serie A')]

#[('obj', 'http://www.wikidata.org/entity/Q1478437'), ('name', 'association football competition')]

#[('obj', 'http://www.wikidata.org/entity/Q15991303'), ('name', 'association football league')]


# I will try to look at national championship Q3270632
queryString = """
SELECT DISTINCT ?p ?pname ?obj ?objname
WHERE {
   # bind something
   wd:Q3270632 ?p ?obj .
   # get the label
   ?p sc:name ?pname.
   ?obj sc:name ?objname.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q1079023'), ('objname', 'championship')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pname', "topic's main category"), ('obj', 'http://www.wikidata.org/entity/Q7481805'), ('objname', 'Category:National championships')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pname', "topic's main template"), ('obj', 'http://www.wikidata.org/entity/Q14358455'), ('objname', 'Template:Infobox national championship')]
3


In [5]:
# I will try to look at association football competition Q1478437
queryString = """
SELECT DISTINCT ?p ?pname ?obj ?objname
WHERE {
   # bind something
   wd:Q1478437 ?p ?obj .
   # get the label
   ?p sc:name ?pname.
   ?obj sc:name ?objname.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q16510064'), ('objname', 'sporting event')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q1079023'), ('objname', 'championship')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pname', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q2736'), ('objname', 'association football')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q500834'), ('objname', 'tournament')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pname', "topic's main category"), ('obj', 'http://www.wikidata.org/entity/Q7015591'), ('objname', 'Category:Association football competitions')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pname', "topic's main template"), ('obj', 'http://www.wikidata.org/entity/Q6284252'), ('objname', 'Temp

In [7]:
# I will try to look at association football league Q15991303
queryString = """
SELECT DISTINCT ?p ?pname ?obj ?objname
WHERE {
   # bind something
   wd:Q15991303 ?p ?obj .
   # get the label
   ?p sc:name ?pname.
   ?obj sc:name ?objname.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P2499'), ('objname', 'league level above')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P2500'), ('objname', 'league level below')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P3983'), ('objname', 'sports league level')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pname', 'different from'), ('obj', 'http://www.wikidata.org/entity/Q26887310'), ('objname', 'association football team season')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P17'), ('objname', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pname', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q273

In [12]:
# look at sport league level P3983
queryString = """
SELECT DISTINCT ?s ?sname ?p ?pname
WHERE {
   # bind something
   ?s ?p wd:P3983.
   # get the label
   ?s sc:name ?sname.
   ?p sc:name ?pname.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q623109'), ('sname', 'sports league'), ('p', 'http://www.wikidata.org/prop/direct/P1687'), ('pname', 'Wikidata property')]
[('s', 'http://www.wikidata.org/entity/Q15991303'), ('sname', 'association football league'), ('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type')]
[('s', 'http://www.wikidata.org/entity/Q106314805'), ('sname', 'volleyball league'), ('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type')]
[('s', 'http://www.wikidata.org/entity/Q75179210'), ('sname', 'handball league'), ('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type')]
[('s', 'http://www.wikidata.org/entity/P2500'), ('sname', 'league level below'), ('p', 'http://www.wikidata.org/prop/direct/P1659'), ('pname', 'see also')]
[('s', 'http://www.wikidata.org/entity/P2499'), ('sname', 'league level above'), ('p', 'http://www.wikidata.org/prop/direct/P1659'), ('pname

In [13]:
# look at sport league level P3983
queryString = """
SELECT DISTINCT ?p ?pname ?obj ?objname
WHERE {
   # bind something
   wd:P3983 ?p ?obj.
   # get the label
   ?obj sc:name ?objname.
   ?p sc:name ?pname.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q28106586'), ('objname', 'Wikidata property related to sport events')]
[('p', 'http://www.wikidata.org/prop/direct/P1659'), ('pname', 'see also'), ('obj', 'http://www.wikidata.org/entity/P2500'), ('objname', 'league level below')]
[('p', 'http://www.wikidata.org/prop/direct/P1659'), ('pname', 'see also'), ('obj', 'http://www.wikidata.org/entity/P2499'), ('objname', 'league level above')]
[('p', 'http://www.wikidata.org/prop/direct/P1659'), ('pname', 'see also'), ('obj', 'http://www.wikidata.org/entity/P6587'), ('objname', 'league system')]
[('p', 'http://www.wikidata.org/prop/direct/P1629'), ('pname', 'Wikidata item of this property'), ('obj', 'http://www.wikidata.org/entity/Q623109'), ('objname', 'sports league')]
[('p', 'http://www.wikidata.org/prop/direct/P1855'), ('pname', 'Wikidata property example'), ('obj', 'http://www.wikidata.org/entity/Q9448'), ('objnam

In [4]:
# I'll try to retrieve the top leagues from nodes that has no league level above P2499 with FILTER NOT EXIST { ... }
# I'll start by retrieving all the europeans countries
# Then for each country, retrieve all its leagues
# Among all the leagues retieved, keep only the one that hasn't league level above P2499

#Q46 europe P30 continent

queryString = """
SELECT DISTINCT ?europeanCountries ?countryName
WHERE {
   # bind something
   ?europeanCountries wdt:P30 wd:Q46 .
   # get the label
   ?europeanCountries sc:name ?countryName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('europeanCountries', 'http://www.wikidata.org/entity/Q334124'), ('countryName', 'Remiremont Abbey')]
[('europeanCountries', 'http://www.wikidata.org/entity/Q1985685'), ('countryName', 'Velká Deštná')]
[('europeanCountries', 'http://www.wikidata.org/entity/Q3674986'), ('countryName', 'Chinatowns in Europe')]
[('europeanCountries', 'http://www.wikidata.org/entity/Q688859'), ('countryName', 'Duchy of Bouillon')]
[('europeanCountries', 'http://www.wikidata.org/entity/Q77120'), ('countryName', 'Eurasian Steppe')]
[('europeanCountries', 'http://www.wikidata.org/entity/Q881713'), ('countryName', 'Blaník')]
[('europeanCountries', 'http://www.wikidata.org/entity/Q12221030'), ('countryName', 'Tábor')]
[('europeanCountries', 'http://www.wikidata.org/entity/Q15853649'), ('countryName', 'Artung')]
[('europeanCountries', 'http://www.wikidata.org/entity/Q23656451'), ('countryName', 'Seehorn (Davos)')]
[('europeanCountries', 'http://www.wikidata.org/entity/Q461361'), ('countryName', 'Amblève

In [5]:
# ?club wdt:P17 ?country .
queryString = """
SELECT DISTINCT ?europeanCountries ?countryName
WHERE {
   # bind something
   ?europeanCountries wdt:P17 wd:Q46 .
   # get the label
   ?europeanCountries sc:name ?countryName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('europeanCountries', 'http://www.wikidata.org/entity/Q39056474'), ('countryName', '2017–18 EHF Challenge Cup')]
[('europeanCountries', 'http://www.wikidata.org/entity/Q3734778'), ('countryName', 'Euroleague MVP')]
[('europeanCountries', 'http://www.wikidata.org/entity/Q1274874'), ('countryName', '2009–10 EHF Challenge Cup')]
[('europeanCountries', 'http://www.wikidata.org/entity/Q682675'), ('countryName', 'EHF European Cup')]
[('europeanCountries', 'http://www.wikidata.org/entity/Q722072'), ('countryName', 'Ozzfest')]
[('europeanCountries', 'http://www.wikidata.org/entity/Q99777894'), ('countryName', 'European Association of Law & Economics')]
[('europeanCountries', 'http://www.wikidata.org/entity/Q19321830'), ('countryName', 'Euroleague Basketball Legend')]
[('europeanCountries', 'http://www.wikidata.org/entity/Q285320'), ('countryName', 'Fat Thursday')]
[('europeanCountries', 'http://www.wikidata.org/entity/Q1550634'), ('countryName', 'VH1 Europe')]
[('europeanCountries', '

In [6]:
# I'll look at Italy Q38 predicates with obj europe
queryString = """
SELECT DISTINCT ?p ?pname
WHERE {
   # bind something
   wd:Q38 ?p wd:Q46 .
   # get the label
   ?p sc:name ?pname .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P30'), ('pname', 'continent')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pname', 'part of')]
2


In [8]:
# I'll look at Italy Q38 predicates in general
queryString = """
SELECT DISTINCT ?p ?pname ?obj ?objname
WHERE {
   # bind something
   wd:Q38 ?p ?obj .
   # get the label
   ?p sc:name ?pname .
   ?obj sc:name ?objname .
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P463'), ('pname', 'member of'), ('obj', 'http://www.wikidata.org/entity/Q488981'), ('objname', 'European Bank for Reconstruction and Development')]
[('p', 'http://www.wikidata.org/prop/direct/P2936'), ('pname', 'language used'), ('obj', 'http://www.wikidata.org/entity/Q33973'), ('objname', 'Sicilian')]
[('p', 'http://www.wikidata.org/prop/direct/P2184'), ('pname', 'history of topic'), ('obj', 'http://www.wikidata.org/entity/Q7791'), ('objname', 'history of Italy')]
[('p', 'http://www.wikidata.org/prop/direct/P163'), ('pname', 'flag'), ('obj', 'http://www.wikidata.org/entity/Q42876'), ('objname', 'flag of Italy')]
[('p', 'http://www.wikidata.org/prop/direct/P150'), ('pname', 'contains administrative territorial entity'), ('obj', 'http://www.wikidata.org/entity/Q1273'), ('objname', 'Tuscany')]
[('p', 'http://www.wikidata.org/prop/direct/P2936'), ('pname', 'language used'), ('obj', 'http://www.wikidata.org/entity/Q33845'), ('objname', 'N

In [10]:
# in order to get only countries, I have to match predicates P31 instance of Q6256 country
# ?club wdt:P17 ?country .
queryString = """
SELECT DISTINCT ?europeanCountries ?countryName
WHERE {
   # bind something
   ?europeanCountries wdt:P31 wd:Q6256 ;
               wdt:P30 wd:Q46 .
   # get the label
   ?europeanCountries sc:name ?countryName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('europeanCountries', 'http://www.wikidata.org/entity/Q756617'), ('countryName', 'Danish Realm')]
[('europeanCountries', 'http://www.wikidata.org/entity/Q907112'), ('countryName', 'Transnistria')]
[('europeanCountries', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France')]
[('europeanCountries', 'http://www.wikidata.org/entity/Q145'), ('countryName', 'United Kingdom')]
[('europeanCountries', 'http://www.wikidata.org/entity/Q184'), ('countryName', 'Belarus')]
[('europeanCountries', 'http://www.wikidata.org/entity/Q20'), ('countryName', 'Norway')]
[('europeanCountries', 'http://www.wikidata.org/entity/Q211'), ('countryName', 'Latvia')]
[('europeanCountries', 'http://www.wikidata.org/entity/Q213'), ('countryName', 'Czech Republic')]
[('europeanCountries', 'http://www.wikidata.org/entity/Q214'), ('countryName', 'Slovakia')]
[('europeanCountries', 'http://www.wikidata.org/entity/Q218'), ('countryName', 'Romania')]
[('europeanCountries', 'http://www.wikidata.org/entity/

In [13]:
# now for each european country I'll retrieve all its leagues that are instance of (P31) association football league Q15991303
queryString = """
SELECT DISTINCT ?europeanCountry ?countryName ?league ?leagueName
WHERE {
   # bind something
   ?europeanCountry wdt:P31 wd:Q6256 ;
               wdt:P30 wd:Q46 .
   ?league wdt:P17 ?europeanCountries ;
            wdt:P31 wd:Q15991303 .
   # get the label
   ?europeanCountry sc:name ?countryName .
   ?league sc:name ?leagueName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('europeanCountry', 'http://www.wikidata.org/entity/Q229'), ('countryName', 'Cyprus'), ('league', 'http://www.wikidata.org/entity/Q304228'), ('leagueName', 'Uzbekistan Pro League')]
[('europeanCountry', 'http://www.wikidata.org/entity/Q229'), ('countryName', 'Cyprus'), ('league', 'http://www.wikidata.org/entity/Q2950655'), ('leagueName', 'Ligue 2')]
[('europeanCountry', 'http://www.wikidata.org/entity/Q229'), ('countryName', 'Cyprus'), ('league', 'http://www.wikidata.org/entity/Q2951924'), ('leagueName', "Moldovan women's football championship")]
[('europeanCountry', 'http://www.wikidata.org/entity/Q229'), ('countryName', 'Cyprus'), ('league', 'http://www.wikidata.org/entity/Q973312'), ('leagueName', 'Championnat National 3')]
[('europeanCountry', 'http://www.wikidata.org/entity/Q229'), ('countryName', 'Cyprus'), ('league', 'http://www.wikidata.org/entity/Q2792041'), ('leagueName', '3. divisjon')]
[('europeanCountry', 'http://www.wikidata.org/entity/Q229'), ('countryName', 'Cy

In [17]:
# I'll try to retrieve the top leagues from nodes that has no league level above P2499 with FILTER NOT EXIST { ... }
queryString = """
SELECT DISTINCT ?europeanCountry ?countryName ?league ?leagueName
WHERE {
   # bind something
   ?europeanCountry wdt:P31 wd:Q6256 ;
               wdt:P30 wd:Q46 .
   ?league wdt:P17 ?europeanCountry ;
            wdt:P31 wd:Q15991303 .
   # get the label
   ?europeanCountry sc:name ?countryName .
   ?league sc:name ?leagueName .
   #FILTER NOT EXIST {?league wdt:P2499 ?obj}.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('europeanCountry', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France'), ('league', 'http://www.wikidata.org/entity/Q973312'), ('leagueName', 'Championnat National 3')]
[('europeanCountry', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France'), ('league', 'http://www.wikidata.org/entity/Q3240443'), ('leagueName', 'Ligue de la Méditerranée de football')]
[('europeanCountry', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France'), ('league', 'http://www.wikidata.org/entity/Q891072'), ('leagueName', 'French Guiana Championnat National')]
[('europeanCountry', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France'), ('league', 'http://www.wikidata.org/entity/Q1061291'), ('leagueName', 'Championnat National 2')]
[('europeanCountry', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France'), ('league', 'http://www.wikidata.org/entity/Q600302'), ('leagueName', 'New Caledonia Division Honneur')]
[('europeanCountry', 'http://www.wik

In [23]:
# I'll try to retrieve the top leagues from nodes that has no league level above P2499 with FILTER NOT EXIST { ... }
queryString = """
SELECT DISTINCT ?europeanCountry ?countryName ?league ?leagueName
WHERE {
   # bind something
   ?europeanCountry wdt:P31 wd:Q6256 ;
               wdt:P30 wd:Q46 .
   ?league wdt:P17 ?europeanCountry ;
            wdt:P31 wd:Q15991303 .
   # get the label
   ?europeanCountry sc:name ?countryName .
   ?league sc:name ?leagueName .
   FILTER NOT EXISTS {?league wdt:P2499 ?someLeague}.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('europeanCountry', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France'), ('league', 'http://www.wikidata.org/entity/Q3240443'), ('leagueName', 'Ligue de la Méditerranée de football')]
[('europeanCountry', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France'), ('league', 'http://www.wikidata.org/entity/Q891072'), ('leagueName', 'French Guiana Championnat National')]
[('europeanCountry', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France'), ('league', 'http://www.wikidata.org/entity/Q600302'), ('leagueName', 'New Caledonia Division Honneur')]
[('europeanCountry', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France'), ('league', 'http://www.wikidata.org/entity/Q2261921'), ('leagueName', 'Réunion Premier League')]
[('europeanCountry', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France'), ('league', 'http://www.wikidata.org/entity/Q13394'), ('leagueName', 'Ligue 1')]
[('europeanCountry', 'http://www.wikidata.org/entity

In [24]:
# I'll try to retrieve the top leagues from nodes that has no league level above P2499 with FILTER NOT EXIST { ... }
queryString = """
SELECT DISTINCT ?europeanCountry ?countryName ?league ?leagueName
WHERE {
   # bind something
   ?europeanCountry wdt:P31 wd:Q6256 ;
               wdt:P30 wd:Q46 .
   ?league wdt:P17 ?europeanCountry ;
            wdt:P31 wd:Q15991303 .
   # get the label
   ?europeanCountry sc:name ?countryName .
   ?league sc:name ?leagueName .
   FILTER EXISTS {?someLeague wdt:P2499 ?league}.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('europeanCountry', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France'), ('league', 'http://www.wikidata.org/entity/Q1061291'), ('leagueName', 'Championnat National 2')]
[('europeanCountry', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France'), ('league', 'http://www.wikidata.org/entity/Q600302'), ('leagueName', 'New Caledonia Division Honneur')]
[('europeanCountry', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France'), ('league', 'http://www.wikidata.org/entity/Q13394'), ('leagueName', 'Ligue 1')]
[('europeanCountry', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France'), ('league', 'http://www.wikidata.org/entity/Q217374'), ('leagueName', 'Ligue 2')]
[('europeanCountry', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France'), ('league', 'http://www.wikidata.org/entity/Q864298'), ('leagueName', 'Championnat National')]
[('europeanCountry', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France'), ('league

In [26]:
# I'll try to retrieve the top leagues from nodes that has no league level above P2499 with FILTER NOT EXIST { ... }
queryString = """
SELECT DISTINCT ?europeanCountry ?countryName ?league ?leagueName
WHERE {
   # bind something
   ?europeanCountry wdt:P31 wd:Q6256 ;
               wdt:P30 wd:Q46 .
   ?league wdt:P17 ?europeanCountry ;
            wdt:P31 wd:Q15991303 .
   # get the label
   ?europeanCountry sc:name ?countryName .
   ?league sc:name ?leagueName .
   FILTER NOT EXISTS {?league wdt:P2499 ?someleague}.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('europeanCountry', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France'), ('league', 'http://www.wikidata.org/entity/Q3240443'), ('leagueName', 'Ligue de la Méditerranée de football')]
[('europeanCountry', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France'), ('league', 'http://www.wikidata.org/entity/Q891072'), ('leagueName', 'French Guiana Championnat National')]
[('europeanCountry', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France'), ('league', 'http://www.wikidata.org/entity/Q600302'), ('leagueName', 'New Caledonia Division Honneur')]
[('europeanCountry', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France'), ('league', 'http://www.wikidata.org/entity/Q2261921'), ('leagueName', 'Réunion Premier League')]
[('europeanCountry', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France'), ('league', 'http://www.wikidata.org/entity/Q13394'), ('leagueName', 'Ligue 1')]
[('europeanCountry', 'http://www.wikidata.org/entity

In [29]:
# try to understand why this doesn't work
queryString = """
SELECT DISTINCT ?league ?leagueName
WHERE {
   # bind something
   ?league wdt:P17 wd:Q38 ;
            wdt:P31 wd:Q15991303 .
   # get the label
   ?league sc:name ?leagueName .
   FILTER NOT EXISTS {?league wdt:P2499 ?someleague}.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('league', 'http://www.wikidata.org/entity/Q15804'), ('leagueName', 'Serie A')]
[('league', 'http://www.wikidata.org/entity/Q1063016'), ('leagueName', 'Prima Categoria')]
[('league', 'http://www.wikidata.org/entity/Q3718185'), ('leagueName', 'Eccellenza Piedmont and Aosta Valley')]
[('league', 'http://www.wikidata.org/entity/Q3229032'), ('leagueName', 'LNPB')]
[('league', 'http://www.wikidata.org/entity/Q3718114'), ('leagueName', 'Eccellenza Lombardy')]
[('league', 'http://www.wikidata.org/entity/Q48851752'), ('leagueName', 'Campionato Primavera 1')]
[('league', 'http://www.wikidata.org/entity/Q86506910'), ('leagueName', 'eSerie A')]
7


In [31]:
# look at Prima Categoria Q1063016 predicates and objs to understand if in fact there's a predicate that indicates the level league above
queryString = """
SELECT DISTINCT ?p ?pname ?obj ?objname
WHERE {
   # bind something
   wd:Q1063016 ?p ?obj .
   # get the label
   ?p sc:name ?pname .
   ?obj sc:name ?objname .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P664'), ('pname', 'organizer'), ('obj', 'http://www.wikidata.org/entity/Q201897'), ('objname', 'Italian Football Federation')]
[('p', 'http://www.wikidata.org/prop/direct/P4794'), ('pname', 'season starts'), ('obj', 'http://www.wikidata.org/entity/Q123'), ('objname', 'September')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pname', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q2736'), ('objname', 'association football')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country'), ('obj', 'http://www.wikidata.org/entity/Q38'), ('objname', 'Italy')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q15991303'), ('objname', 'association football league')]
[('p', 'http://www.wikidata.org/prop/direct/P1366'), ('pname', 'replaced by'), ('obj', 'http://www.wikidata.org/entity/Q3911657'), ('objname', 'Prima Divisione')]
6


In [35]:
# Try to look at P3983 sports league level of Serie A
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q15804 wdt:P3983 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [37]:
# Try to look at P3983 sports league level of Premier League Q9448
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q9448 wdt:P3983 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [39]:
# Try to understand if the problem is the label name
queryString = """
SELECT DISTINCT ?obj
WHERE {
   # bind something
   wd:Q15804 wdt:P3983 ?obj .
   # get the label
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', '1')]
1


In [42]:
# now that I understand that the problem with P3983 sports league level property was the label, I will retrieve all the leagues with sports league level 1 of european countries
queryString = """
SELECT DISTINCT ?europeanCountry ?countryName ?league ?leagueName
WHERE {
   # bind something
   ?europeanCountry wdt:P31 wd:Q6256 ;
               wdt:P30 wd:Q46 .
   ?league wdt:P17 ?europeanCountry ;
            wdt:P31 wd:Q15991303 ;
            wdt:P3983 1 .
   # get the label
   ?europeanCountry sc:name ?countryName .
   ?league sc:name ?leagueName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('europeanCountry', 'http://www.wikidata.org/entity/Q214'), ('countryName', 'Slovakia'), ('league', 'http://www.wikidata.org/entity/Q238113'), ('leagueName', 'Slovak Super Liga')]
[('europeanCountry', 'http://www.wikidata.org/entity/Q35'), ('countryName', 'Denmark'), ('league', 'http://www.wikidata.org/entity/Q739479'), ('leagueName', 'Faroe Islands Premier League')]
[('europeanCountry', 'http://www.wikidata.org/entity/Q33'), ('countryName', 'Finland'), ('league', 'http://www.wikidata.org/entity/Q220875'), ('leagueName', 'Veikkausliiga')]
[('europeanCountry', 'http://www.wikidata.org/entity/Q45'), ('countryName', 'Portugal'), ('league', 'http://www.wikidata.org/entity/Q182994'), ('leagueName', 'Liga Portugal')]
[('europeanCountry', 'http://www.wikidata.org/entity/Q34'), ('countryName', 'Sweden'), ('league', 'http://www.wikidata.org/entity/Q202243'), ('leagueName', 'Allsvenskan')]
[('europeanCountry', 'http://www.wikidata.org/entity/Q36'), ('countryName', 'Poland'), ('league', 

Final query for this task

In [43]:
# write your final query
# now that I understand that the problem with P3983 sports league level property was the label, I will retrieve all the leagues with sports league level 1 of european countries
queryString = """
SELECT DISTINCT ?europeanCountry ?countryName ?league ?leagueName
WHERE {
   # bind something
   ?europeanCountry wdt:P31 wd:Q6256 ;
               wdt:P30 wd:Q46 .
   ?league wdt:P17 ?europeanCountry ;
            wdt:P31 wd:Q15991303 ;
            wdt:P3983 1 .
   # get the label
   ?europeanCountry sc:name ?countryName .
   ?league sc:name ?leagueName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('europeanCountry', 'http://www.wikidata.org/entity/Q214'), ('countryName', 'Slovakia'), ('league', 'http://www.wikidata.org/entity/Q238113'), ('leagueName', 'Slovak Super Liga')]
[('europeanCountry', 'http://www.wikidata.org/entity/Q35'), ('countryName', 'Denmark'), ('league', 'http://www.wikidata.org/entity/Q739479'), ('leagueName', 'Faroe Islands Premier League')]
[('europeanCountry', 'http://www.wikidata.org/entity/Q33'), ('countryName', 'Finland'), ('league', 'http://www.wikidata.org/entity/Q220875'), ('leagueName', 'Veikkausliiga')]
[('europeanCountry', 'http://www.wikidata.org/entity/Q45'), ('countryName', 'Portugal'), ('league', 'http://www.wikidata.org/entity/Q182994'), ('leagueName', 'Liga Portugal')]
[('europeanCountry', 'http://www.wikidata.org/entity/Q34'), ('countryName', 'Sweden'), ('league', 'http://www.wikidata.org/entity/Q202243'), ('leagueName', 'Allsvenskan')]
[('europeanCountry', 'http://www.wikidata.org/entity/Q36'), ('countryName', 'Poland'), ('league', 

## Task 7
Identify the BGP of the FIFA World Cup

In [3]:
# write your queries
# I'll start by looking at 2018 FIFA World cup Q170645 founded on the third query of task 1.

queryString = """
SELECT DISTINCT ?p ?pname ?obj ?objname
WHERE {
   # bind something
   wd:Q170645 ?p ?obj .
   # get the label
   ?p sc:name ?pname.
   ?obj sc:name ?objname.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P3967'), ('pname', 'final event'), ('obj', 'http://www.wikidata.org/entity/Q31043671'), ('objname', '2018 FIFA World Cup Final')]
[('p', 'http://www.wikidata.org/prop/direct/P664'), ('pname', 'organizer'), ('obj', 'http://www.wikidata.org/entity/Q253414'), ('objname', 'FIFA')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('pname', 'location'), ('obj', 'http://www.wikidata.org/entity/Q202163'), ('objname', 'Luzhniki Stadium')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country'), ('obj', 'http://www.wikidata.org/entity/Q159'), ('objname', 'Russia')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q27020041'), ('objname', 'sports season')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pname', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q2736'), ('objname', 'association football')]
[('p', 'http://www.wikidata.org/prop/direct/P1346

Final query for this task

In [4]:
# write your final query
# I will look at the property P3450 sports season of league or competition of the 2018 FIFA World Cup Q170645
queryString = """
SELECT DISTINCT ?obj ?objname
WHERE {
   # bind something
   wd:Q170645 wdt:P3450 ?obj .
   # get the label
   ?obj sc:name ?objname.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q19317'), ('objname', 'FIFA World Cup')]
1


## Task 8
Consider countries who participated at the FIFA World Cup more than 10 times. Return these countries with the number of apparitions (the result set must be triples of country IRI, label and #apparitions).

In [45]:
# write your queries
# I will start by retrieving all the FIFA World Cup editions using the previous task's query.
# I will look at the property P3450 sports season of league or competition of the 2018 FIFA World Cup Q170645
queryString = """
SELECT DISTINCT ?edition ?editionName
WHERE {
   # bind something
   ?edition wdt:P3450 wd:Q19317 .
   # get the label
   ?edition sc:name ?editionName.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('edition', 'http://www.wikidata.org/entity/Q101751'), ('editionName', '1994 FIFA World Cup')]
[('edition', 'http://www.wikidata.org/entity/Q131591'), ('editionName', '1938 FIFA World Cup')]
[('edition', 'http://www.wikidata.org/entity/Q132664'), ('editionName', '1970 FIFA World Cup')]
[('edition', 'http://www.wikidata.org/entity/Q48432'), ('editionName', '1930 FIFA World Cup')]
[('edition', 'http://www.wikidata.org/entity/Q170645'), ('editionName', '2018 FIFA World Cup')]
[('edition', 'http://www.wikidata.org/entity/Q176883'), ('editionName', '2010 FIFA World Cup')]
[('edition', 'http://www.wikidata.org/entity/Q37285'), ('editionName', '2006 FIFA World Cup')]
[('edition', 'http://www.wikidata.org/entity/Q46934'), ('editionName', '1982 FIFA World Cup')]
[('edition', 'http://www.wikidata.org/entity/Q46938'), ('editionName', '1986 FIFA World Cup')]
[('edition', 'http://www.wikidata.org/entity/Q46943'), ('editionName', '1978 FIFA World Cup')]
[('edition', 'http://www.wikidata.org

In [46]:
# just order the results for a better overview
queryString = """
SELECT DISTINCT ?edition ?editionName
WHERE {
   # bind something
   ?edition wdt:P3450 wd:Q19317 .
   # get the label
   ?edition sc:name ?editionName.
}
ORDER BY ?editionName
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('edition', 'http://www.wikidata.org/entity/Q48432'), ('editionName', '1930 FIFA World Cup')]
[('edition', 'http://www.wikidata.org/entity/Q1477177'), ('editionName', '1934 FIFA World Cup')]
[('edition', 'http://www.wikidata.org/entity/Q131591'), ('editionName', '1938 FIFA World Cup')]
[('edition', 'http://www.wikidata.org/entity/Q203851'), ('editionName', '1942 FIFA World Cup')]
[('edition', 'http://www.wikidata.org/entity/Q65273310'), ('editionName', '1946 FIFA World Cup')]
[('edition', 'http://www.wikidata.org/entity/Q132515'), ('editionName', '1950 FIFA World Cup')]
[('edition', 'http://www.wikidata.org/entity/Q131075'), ('editionName', '1954 FIFA World Cup')]
[('edition', 'http://www.wikidata.org/entity/Q132533'), ('editionName', '1958 FIFA World Cup')]
[('edition', 'http://www.wikidata.org/entity/Q160813'), ('editionName', '1962 FIFA World Cup')]
[('edition', 'http://www.wikidata.org/entity/Q134202'), ('editionName', '1966 FIFA World Cup')]
[('edition', 'http://www.wikid

In [49]:
# Now I'll start to retrieve all the partecipants for each edition
# Here, I'll look at the predicates and objs of 2018 FIFA World Cup Q170645, supposing that all other editions have the same structure
queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName
WHERE {
   # bind something
   wd:Q170645 ?p ?obj .
   
   # get the label
   ?p sc:name ?pName.
   ?obj sc:name ?objName.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P3967'), ('pName', 'final event'), ('obj', 'http://www.wikidata.org/entity/Q31043671'), ('objName', '2018 FIFA World Cup Final')]
[('p', 'http://www.wikidata.org/prop/direct/P664'), ('pName', 'organizer'), ('obj', 'http://www.wikidata.org/entity/Q253414'), ('objName', 'FIFA')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('pName', 'location'), ('obj', 'http://www.wikidata.org/entity/Q202163'), ('objName', 'Luzhniki Stadium')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country'), ('obj', 'http://www.wikidata.org/entity/Q159'), ('objName', 'Russia')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q27020041'), ('objName', 'sports season')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q2736'), ('objName', 'association football')]
[('p', 'http://www.wikidata.org/prop/direct/P1346

In [50]:
# With predicate P1923 participating team I'll retrieve all the partecipants to that edition
queryString = """
SELECT DISTINCT ?obj ?objName
WHERE {
   # bind something
   wd:Q170645 wdt:P1923 ?obj .
   
   # get the label
   ?obj sc:name ?objName.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q45101586'), ('objName', 'Australia at the 2018 FIFA World Cup')]
[('obj', 'http://www.wikidata.org/entity/Q43196508'), ('objName', 'Uruguay at the 2018 FIFA World Cup\u200e')]
[('obj', 'http://www.wikidata.org/entity/Q43196538'), ('objName', 'Brazil at the 2018 FIFA World Cup\u200e')]
[('obj', 'http://www.wikidata.org/entity/Q43122212'), ('objName', 'Iceland at the 2018 FIFA World Cup')]
[('obj', 'http://www.wikidata.org/entity/Q45905387'), ('objName', 'Croatia at the 2018 FIFA World Cup')]
[('obj', 'http://www.wikidata.org/entity/Q43197686'), ('objName', 'Panama at the 2018 FIFA World Cup')]
[('obj', 'http://www.wikidata.org/entity/Q43249937'), ('objName', 'France at the 2018 FIFA World Cup\u200e')]
[('obj', 'http://www.wikidata.org/entity/Q44959733'), ('objName', 'Egypt at the 2018 FIFA World Cup\u200e')]
[('obj', 'http://www.wikidata.org/entity/Q43122091'), ('objName', 'Germany at the 2018 FIFA World Cup')]
[('obj', 'http://www.wikid

In [51]:
# Now I'll look at the predicates and objs of Q43196530 Argentina at the 2018 FIFA World Cup to see if I can retrieve the country
queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName
WHERE {
   # bind something
   wd:Q43196530 ?p ?obj .
   
   # get the label
   ?p sc:name ?pName.
   ?obj sc:name ?objName.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('pName', 'facet of'), ('obj', 'http://www.wikidata.org/entity/Q170645'), ('objName', '2018 FIFA World Cup')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q2736'), ('objName', 'association football')]
[('p', 'http://www.wikidata.org/prop/direct/P1532'), ('pName', 'country for sport'), ('obj', 'http://www.wikidata.org/entity/Q414'), ('objName', 'Argentina')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country'), ('obj', 'http://www.wikidata.org/entity/Q414'), ('objName', 'Argentina')]
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('pName', 'facet of'), ('obj', 'http://www.wikidata.org/entity/Q4789306'), ('objName', 'Argentina at the FIFA World Cup')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q4789306'), ('objName', 'Argentina at the FIFA World Cup')]
[('p', 'http

In [53]:
# Now I'll put everything together: starting from the 2018 edition, retrieve all the countries that partecipated
queryString = """
SELECT DISTINCT ?country ?countryName
WHERE {
   # bind something
   wd:Q170645 wdt:P1923 ?partecipatingTeam .
   ?partecipatingTeam wdt:P17 ?country .
   
   # get the label
   ?country sc:name ?countryName.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q1041'), ('countryName', 'Senegal')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('countryName', 'United Kingdom')]
[('country', 'http://www.wikidata.org/entity/Q155'), ('countryName', 'Brazil')]
[('country', 'http://www.wikidata.org/entity/Q29'), ('countryName', 'Spain')]
[('country', 'http://www.wikidata.org/entity/Q34'), ('countryName', 'Sweden')]
[('country', 'http://www.wikidata.org/entity/Q408'), ('countryName', 'Australia')]
[('country', 'http://www.wikidata.org/entity/Q414'), ('countryName', 'Argentina')]
[('country', 'http://www.wikidata.org/entity/Q419'), ('countryName', 'Peru')]
[('country', 'http://www.wikidata.org/entity/Q45'), ('countryName', 'Portugal')]
[('country', 'http://www.wikidata.org/entity/Q739'), ('countryName', 'Colombia')]
[('country', 'http://www.wikidata.org/entity/Q77'), ('countryName', 'Uruguay')]
[('country', 'http://www.wikidata.org/entity/Q79'), ('countryName', 'Egypt')]
[('country', 'http:/

In [54]:
# Now I'll look at the predicates and objs of Q45590884 Poland at the 2018 FIFA World Cup to understand why Poland didn't appear on the previous results
queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName
WHERE {
   # bind something
   wd:Q45590884 ?p ?obj .
   
   # get the label
   ?p sc:name ?pName.
   ?obj sc:name ?objName.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('pName', 'facet of'), ('obj', 'http://www.wikidata.org/entity/Q7208923'), ('objName', 'Poland at the FIFA World Cup')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q7208923'), ('objName', 'Poland at the FIFA World Cup')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of'), ('obj', 'http://www.wikidata.org/entity/Q170645'), ('objName', '2018 FIFA World Cup')]
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('pName', 'facet of'), ('obj', 'http://www.wikidata.org/entity/Q170645'), ('objName', '2018 FIFA World Cup')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q2736'), ('objName', 'association football')]
[('p', 'http://www.wikidata.org/prop/direct/P1532'), ('pName', 'country for sport'), ('obj', 'http://www.wikidata.org/entity/Q36'), ('objName', 'Poland')]
[('p', '

In [55]:
# probably is better to use country for sport P1532 instead of country P17
queryString = """
SELECT DISTINCT ?country ?countryName
WHERE {
   # bind something
   wd:Q170645 wdt:P1923 ?partecipatingTeam .
   ?partecipatingTeam wdt:P1532 ?country .
   
   # get the label
   ?country sc:name ?countryName.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q1028'), ('countryName', 'Morocco')]
[('country', 'http://www.wikidata.org/entity/Q1033'), ('countryName', 'Nigeria')]
[('country', 'http://www.wikidata.org/entity/Q1041'), ('countryName', 'Senegal')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France')]
[('country', 'http://www.wikidata.org/entity/Q155'), ('countryName', 'Brazil')]
[('country', 'http://www.wikidata.org/entity/Q159'), ('countryName', 'Russia')]
[('country', 'http://www.wikidata.org/entity/Q17'), ('countryName', 'Japan')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('countryName', 'Germany')]
[('country', 'http://www.wikidata.org/entity/Q189'), ('countryName', 'Iceland')]
[('country', 'http://www.wikidata.org/entity/Q21'), ('countryName', 'England')]
[('country', 'http://www.wikidata.org/entity/Q224'), ('countryName', 'Croatia')]
[('country', 'http://www.wikidata.org/entity/Q29'), ('countryName', 'Spain')]
[('country', 'http://www.wi

In [57]:
# just order previous query
queryString = """
SELECT DISTINCT ?country ?countryName
WHERE {
   # bind something
   wd:Q170645 wdt:P1923 ?partecipatingTeam .
   ?partecipatingTeam wdt:P1532 ?country .
   
   # get the label
   ?country sc:name ?countryName.
}
ORDER BY ?countryName
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q414'), ('countryName', 'Argentina')]
[('country', 'http://www.wikidata.org/entity/Q408'), ('countryName', 'Australia')]
[('country', 'http://www.wikidata.org/entity/Q31'), ('countryName', 'Belgium')]
[('country', 'http://www.wikidata.org/entity/Q155'), ('countryName', 'Brazil')]
[('country', 'http://www.wikidata.org/entity/Q739'), ('countryName', 'Colombia')]
[('country', 'http://www.wikidata.org/entity/Q800'), ('countryName', 'Costa Rica')]
[('country', 'http://www.wikidata.org/entity/Q224'), ('countryName', 'Croatia')]
[('country', 'http://www.wikidata.org/entity/Q35'), ('countryName', 'Denmark')]
[('country', 'http://www.wikidata.org/entity/Q79'), ('countryName', 'Egypt')]
[('country', 'http://www.wikidata.org/entity/Q21'), ('countryName', 'England')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('countryName', 'Germany')]
[('country', 'http:/

In [56]:
# With predicate P1923 participating team I'll retrieve all the partecipants to that edition
queryString = """
SELECT DISTINCT ?obj ?objName
WHERE {
   # bind something
   wd:Q170645 wdt:P1923 ?obj .
   
   # get the label
   ?obj sc:name ?objName.
}
ORDER BY ?objName
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q43196530'), ('objName', 'Argentina at the 2018 FIFA World Cup')]
[('obj', 'http://www.wikidata.org/entity/Q45101586'), ('objName', 'Australia at the 2018 FIFA World Cup')]
[('obj', 'http://www.wikidata.org/entity/Q43122121'), ('objName', 'Belgium at the 2018 FIFA World Cup')]
[('obj', 'http://www.wikidata.org/entity/Q43196538'), ('objName', 'Brazil at the 2018 FIFA World Cup\u200e')]
[('obj', 'http://www.wikidata.org/entity/Q43196480'), ('objName', 'Colombia at the 2018 FIFA World Cup')]
[('obj', 'http://www.wikidata.org/entity/Q46265248'), ('objName', 'Costa Rica at the 2018 FIFA World Cup')]
[('obj', 'http://www.wikidata.org/entity/Q45905387'), ('objName', 'Croatia at the 2018 FIFA World Cup')]
[('obj', 'http://www.wikidata.org/entity/Q45186603'), ('objName', 'Denmark at the 2018 FIFA World Cup')]
[('obj', 'http://www.wikidata.org/entity/Q44959733'), ('objName', 'Egypt at the 2018 FIFA World Cup\u200e')]
[('obj', 'http://www.wikidata.

In [58]:
# Now I'll look at the predicates and objs of Q44959591 Tunisia at the 2018 FIFA World Cup to understand why Tunisia didn't appear on the previous results
queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName
WHERE {
   # bind something
   wd:Q44959591 ?p ?obj .
   
   # get the label
   ?p sc:name ?pName.
   ?obj sc:name ?objName.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('pName', 'facet of'), ('obj', 'http://www.wikidata.org/entity/Q170645'), ('objName', '2018 FIFA World Cup')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q2736'), ('objName', 'association football')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country'), ('obj', 'http://www.wikidata.org/entity/Q948'), ('objName', 'Tunisia')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q46135307'), ('objName', 'nation at sport competition')]
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('pName', 'facet of'), ('obj', 'http://www.wikidata.org/entity/Q16858481'), ('objName', 'Tunisia at the FIFA World Cup')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q54812340'), ('objName', '2018 FIFA World Cup team')]
[('p'

In [61]:
# ON THE PREVIOUS QUERY I NOTICE AN INCONSISTENCY of the snapshot: Tunisia doesn't have the predicate P1532 country for sport, while in the real wikidata there's.
# I'll try another path to reach the results, using Q54812340 2018 FIFA World Cup team

queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName
WHERE {
   # bind something
   wd:Q54812340 ?p ?obj .
   
   # get the label
   ?p sc:name ?pName.
   ?obj sc:name ?objName.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('pName', 'participant in'), ('obj', 'http://www.wikidata.org/entity/Q170645'), ('objName', '2018 FIFA World Cup')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q2736'), ('objName', 'association football')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q46135307'), ('objName', 'nation at sport competition')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q1477115'), ('objName', 'FIFA World Cup team')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('pName', 'follows'), ('obj', 'http://www.wikidata.org/entity/Q20436354'), ('objName', '2014 FIFA World Cup team')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pName', "topic's main category"), ('obj', 'http://www.wikidata.org/entity/Q31529939'), ('objName', 'Cat

In [60]:
# I'll try another path to reach the results, using Q54812340 2018 FIFA World Cup team

queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName
WHERE {
   # bind something
   ?s ?p wd:Q54812340 .
   
   # get the label
   ?p sc:name ?pName.
   ?s sc:name ?sName.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q20436354'), ('sName', '2014 FIFA World Cup team'), ('p', 'http://www.wikidata.org/prop/direct/P156'), ('pName', 'followed by')]
[('s', 'http://www.wikidata.org/entity/Q45101586'), ('sName', 'Australia at the 2018 FIFA World Cup'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q43196508'), ('sName', 'Uruguay at the 2018 FIFA World Cup\u200e'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q43196538'), ('sName', 'Brazil at the 2018 FIFA World Cup\u200e'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q43122212'), ('sName', 'Iceland at the 2018 FIFA World Cup'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q45905387'), ('sName', 'Croatia at the 2018 FIFA World Cup'), ('

In [62]:
# I'll try with Q45056532 2018 FIFA World Cup squads
queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName
WHERE {
   # bind something
   wd:Q45056532 ?p ?obj .
   
   # get the label
   ?p sc:name ?pName.
   ?obj sc:name ?objName.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q13406463'), ('objName', 'Wikimedia list article')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q2736'), ('objName', 'association football')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('pName', 'follows'), ('obj', 'http://www.wikidata.org/entity/Q16658998'), ('objName', '2014 FIFA World Cup squads')]
[('p', 'http://www.wikidata.org/prop/direct/P360'), ('pName', 'is a list of'), ('obj', 'http://www.wikidata.org/entity/Q54812340'), ('objName', '2018 FIFA World Cup team')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pName', "topic's main category"), ('obj', 'http://www.wikidata.org/entity/Q53616371'), ('objName', 'Category:2018 FIFA World Cup players')]
[('p', 'http://www.wikidata.org/prop/direct/P1754'), ('pName', 'category related to list'), ('obj', 'http://www.wikidata.org/ent

In [64]:
# I'll try with Q45056532 2018 FIFA World Cup squads
queryString = """
SELECT DISTINCT ?p ?pName ?obj
WHERE {
   # bind something
   wd:Q45056532 ?p ?obj .
   
   # get the label
   ?p sc:name ?pName.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('pName', 'follows'), ('obj', 'http://www.wikidata.org/entity/Q16658998')]
[('p', 'http://www.wikidata.org/prop/direct/P1754'), ('pName', 'category related to list'), ('obj', 'http://www.wikidata.org/entity/Q31529939')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q13406463')]
[('p', 'http://www.wikidata.org/prop/direct/P360'), ('pName', 'is a list of'), ('obj', 'http://www.wikidata.org/entity/Q54812340')]
[('p', 'http://www.wikidata.org/prop/direct/P582'), ('pName', 'end time'), ('obj', '2018-01-01T00:00:00Z')]
[('p', 'http://www.wikidata.org/prop/direct/P585'), ('pName', 'point in time'), ('obj', '2018-01-01T00:00:00Z')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q2736')]
[('p', 'http://www.wikidata.org/prop/direct/P856'), ('pName', 'official website'), ('obj', 'https://www.fifa.com/wor

In [70]:
# I understand that the previous query can't work. I'll try to use union in order to retrieve all the countries
queryString = """
SELECT DISTINCT ?country ?countryName
WHERE {
   # bind something
   wd:Q170645 wdt:P1923 ?partecipatingTeam .
   {
       ?partecipatingTeam wdt:P1532 ?country .
   }
   UNION
   {
       ?partecipatingTeam wdt:P17 ?country .
   }
   
   # get the label
   ?country sc:name ?countryName.
}
ORDER BY ?countryName
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q414'), ('countryName', 'Argentina')]
[('country', 'http://www.wikidata.org/entity/Q408'), ('countryName', 'Australia')]
[('country', 'http://www.wikidata.org/entity/Q31'), ('countryName', 'Belgium')]
[('country', 'http://www.wikidata.org/entity/Q155'), ('countryName', 'Brazil')]
[('country', 'http://www.wikidata.org/entity/Q739'), ('countryName', 'Colombia')]
[('country', 'http://www.wikidata.org/entity/Q800'), ('countryName', 'Costa Rica')]
[('country', 'http://www.wikidata.org/entity/Q224'), ('countryName', 'Croatia')]
[('country', 'http://www.wikidata.org/entity/Q35'), ('countryName', 'Denmark')]
[('country', 'http://www.wikidata.org/entity/Q79'), ('countryName', 'Egypt')]
[('country', 'http://www.wikidata.org/entity/Q21'), ('countryName', 'England')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('countryName', 'Germany')]
[('country', 'http:/

In [73]:
# previous query added also Q145 United Kingdom, that I'll try to exclude
queryString = """
SELECT DISTINCT ?country ?countryName
WHERE {
   # bind something
   wd:Q170645 wdt:P1923 ?partecipatingTeam .
   {
       ?partecipatingTeam wdt:P1532 ?country .
   }
   UNION
   {
       ?partecipatingTeam wdt:P17 ?country .
   }
   MINUS
   {
       ?partecipatingTeam wdt:P17 wd:Q145 .
   }
   
   # get the label
   ?country sc:name ?countryName.
}
ORDER BY ?countryName
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q414'), ('countryName', 'Argentina')]
[('country', 'http://www.wikidata.org/entity/Q408'), ('countryName', 'Australia')]
[('country', 'http://www.wikidata.org/entity/Q31'), ('countryName', 'Belgium')]
[('country', 'http://www.wikidata.org/entity/Q155'), ('countryName', 'Brazil')]
[('country', 'http://www.wikidata.org/entity/Q739'), ('countryName', 'Colombia')]
[('country', 'http://www.wikidata.org/entity/Q800'), ('countryName', 'Costa Rica')]
[('country', 'http://www.wikidata.org/entity/Q224'), ('countryName', 'Croatia')]
[('country', 'http://www.wikidata.org/entity/Q35'), ('countryName', 'Denmark')]
[('country', 'http://www.wikidata.org/entity/Q79'), ('countryName', 'Egypt')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('countryName', 'Germany')]
[('country', 'http://www.wikidata.org/entity/Q189'), ('countryName', 'Iceland')]
[('country', 'http:

In [74]:
# previous query exluded also England, that must be included. I'll try to esclude UK in a different way
queryString = """
SELECT DISTINCT ?country ?countryName
WHERE {
   # bind something
   wd:Q170645 wdt:P1923 ?partecipatingTeam .
   {
       ?partecipatingTeam wdt:P1532 ?country .
   }
   UNION
   {
       ?partecipatingTeam wdt:P17 ?country .
   }
   MINUS
   {
       ?partecipatingTeam ?p wd:Q145 .
   }
   
   # get the label
   ?country sc:name ?countryName.
}
ORDER BY ?countryName
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q414'), ('countryName', 'Argentina')]
[('country', 'http://www.wikidata.org/entity/Q408'), ('countryName', 'Australia')]
[('country', 'http://www.wikidata.org/entity/Q31'), ('countryName', 'Belgium')]
[('country', 'http://www.wikidata.org/entity/Q155'), ('countryName', 'Brazil')]
[('country', 'http://www.wikidata.org/entity/Q739'), ('countryName', 'Colombia')]
[('country', 'http://www.wikidata.org/entity/Q800'), ('countryName', 'Costa Rica')]
[('country', 'http://www.wikidata.org/entity/Q224'), ('countryName', 'Croatia')]
[('country', 'http://www.wikidata.org/entity/Q35'), ('countryName', 'Denmark')]
[('country', 'http://www.wikidata.org/entity/Q79'), ('countryName', 'Egypt')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('countryName', 'Germany')]
[('country', 'http://www.wikidata.org/entity/Q189'), ('countryName', 'Iceland')]
[('country', 'http:

In [71]:
queryString = """
SELECT DISTINCT ?country ?countryName
WHERE {
   # bind something
   wd:Q170645 wdt:P1923 ?partecipatingTeam .
   {
       ?partecipatingTeam wdt:P1532 ?country .
   }
   UNION
   {
       ?partecipatingTeam wdt:P17 ?country .
   }
   MINUS
   {
       ?partecipatingTeam wdt:P1532 wd:Q145 .
   }
   
   # get the label
   ?country sc:name ?countryName.
}
ORDER BY ?countryName
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q414'), ('countryName', 'Argentina')]
[('country', 'http://www.wikidata.org/entity/Q408'), ('countryName', 'Australia')]
[('country', 'http://www.wikidata.org/entity/Q31'), ('countryName', 'Belgium')]
[('country', 'http://www.wikidata.org/entity/Q155'), ('countryName', 'Brazil')]
[('country', 'http://www.wikidata.org/entity/Q739'), ('countryName', 'Colombia')]
[('country', 'http://www.wikidata.org/entity/Q800'), ('countryName', 'Costa Rica')]
[('country', 'http://www.wikidata.org/entity/Q224'), ('countryName', 'Croatia')]
[('country', 'http://www.wikidata.org/entity/Q35'), ('countryName', 'Denmark')]
[('country', 'http://www.wikidata.org/entity/Q79'), ('countryName', 'Egypt')]
[('country', 'http://www.wikidata.org/entity/Q21'), ('countryName', 'England')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('countryName', 'Germany')]
[('country', 'http:/

In [72]:
queryString = """
SELECT DISTINCT ?country ?countryName
WHERE {
   # bind something
   wd:Q170645 wdt:P1923 ?partecipatingTeam .
   {
       ?partecipatingTeam wdt:P1532 ?country .
   }
   UNION
   {
       ?partecipatingTeam wdt:P17 ?country .
   }
   
   # get the label
   ?country sc:name ?countryName.
   FILTER (?countryName != "United Kingdom")
}
ORDER BY ?countryName
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q414'), ('countryName', 'Argentina')]
[('country', 'http://www.wikidata.org/entity/Q408'), ('countryName', 'Australia')]
[('country', 'http://www.wikidata.org/entity/Q31'), ('countryName', 'Belgium')]
[('country', 'http://www.wikidata.org/entity/Q155'), ('countryName', 'Brazil')]
[('country', 'http://www.wikidata.org/entity/Q739'), ('countryName', 'Colombia')]
[('country', 'http://www.wikidata.org/entity/Q800'), ('countryName', 'Costa Rica')]
[('country', 'http://www.wikidata.org/entity/Q224'), ('countryName', 'Croatia')]
[('country', 'http://www.wikidata.org/entity/Q35'), ('countryName', 'Denmark')]
[('country', 'http://www.wikidata.org/entity/Q79'), ('countryName', 'Egypt')]
[('country', 'http://www.wikidata.org/entity/Q21'), ('countryName', 'England')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('countryName', 'Germany')]
[('country', 'http:/

In [81]:
# restarting from the 4th query, I'll try to reach all the country from the node 'country at the FIFA World Cup'
queryString = """
SELECT DISTINCT ?obj ?objName
WHERE {
   # bind something
   wd:Q170645 wdt:P1923 ?partecipatingTeam .
   ?partecipatingTeam wdt:P31 ?obj .
   
   # get the label
   ?obj sc:name ?objName.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q16851252'), ('objName', 'Sweden at the FIFA World Cup')]
[('obj', 'http://www.wikidata.org/entity/Q7567692'), ('objName', 'South Korea at the FIFA World Cup')]
[('obj', 'http://www.wikidata.org/entity/Q7208923'), ('objName', 'Poland at the FIFA World Cup')]
[('obj', 'http://www.wikidata.org/entity/Q16851444'), ('objName', 'Switzerland at the FIFA World Cup')]
[('obj', 'http://www.wikidata.org/entity/Q4823685'), ('objName', 'Australia at the FIFA World Cup')]
[('obj', 'http://www.wikidata.org/entity/Q46135307'), ('objName', 'nation at sport competition')]
[('obj', 'http://www.wikidata.org/entity/Q16838718'), ('objName', 'England at the FIFA World Cup')]
[('obj', 'http://www.wikidata.org/entity/Q2693659'), ('objName', 'Japan at the FIFA World Cup')]
[('obj', 'http://www.wikidata.org/entity/Q7381959'), ('objName', 'Russia at the FIFA World Cup')]
[('obj', 'http://www.wikidata.org/entity/Q7572884'), ('objName', 'Spain at the FIFA World Cup'

In [82]:
# I understand that I was doing an error on the minus clause on previous query, so I'll restart from that one.
queryString = """
SELECT DISTINCT ?country ?countryName
WHERE {
   # bind something
   wd:Q170645 wdt:P1923 ?partecipatingTeam .
   {
       ?partecipatingTeam wdt:P1532 ?country .
   }
   UNION
   {
       ?partecipatingTeam wdt:P17 ?country .
       MINUS
       {
           ?partecipatingTeam ?p wd:Q145 .
       }
   }
   
   
   # get the label
   ?country sc:name ?countryName.
}
ORDER BY ?countryName
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q414'), ('countryName', 'Argentina')]
[('country', 'http://www.wikidata.org/entity/Q408'), ('countryName', 'Australia')]
[('country', 'http://www.wikidata.org/entity/Q31'), ('countryName', 'Belgium')]
[('country', 'http://www.wikidata.org/entity/Q155'), ('countryName', 'Brazil')]
[('country', 'http://www.wikidata.org/entity/Q739'), ('countryName', 'Colombia')]
[('country', 'http://www.wikidata.org/entity/Q800'), ('countryName', 'Costa Rica')]
[('country', 'http://www.wikidata.org/entity/Q224'), ('countryName', 'Croatia')]
[('country', 'http://www.wikidata.org/entity/Q35'), ('countryName', 'Denmark')]
[('country', 'http://www.wikidata.org/entity/Q79'), ('countryName', 'Egypt')]
[('country', 'http://www.wikidata.org/entity/Q21'), ('countryName', 'England')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('countryName', 'Germany')]
[('country', 'http:/

In [93]:
# Now I'll try to group by country
queryString = """
SELECT DISTINCT ?country ?countryName
WHERE {
   # bind something
   ?edition wdt:P3450 wd:Q19317 ;
       wdt:P1923 ?partecipatingTeam .
   {
       ?partecipatingTeam wdt:P1532 ?country .
   }
   UNION
   {
       ?partecipatingTeam wdt:P17 ?country .
       MINUS
       {
           ?partecipatingTeam ?p wd:Q145 .
       }
   }
   
   
   # get the label
   ?country sc:name ?countryName.
}
ORDER BY ?countryName
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q262'), ('countryName', 'Algeria')]
[('country', 'http://www.wikidata.org/entity/Q916'), ('countryName', 'Angola')]
[('country', 'http://www.wikidata.org/entity/Q414'), ('countryName', 'Argentina')]
[('country', 'http://www.wikidata.org/entity/Q408'), ('countryName', 'Australia')]
[('country', 'http://www.wikidata.org/entity/Q40'), ('countryName', 'Austria')]
[('country', 'http://www.wikidata.org/entity/Q31'), ('countryName', 'Belgium')]
[('country', 'http://www.wikidata.org/entity/Q750'), ('countryName', 'Bolivia')]
[('country', 'http://www.wikidata.org/entity/Q225'), ('countryName', 'Bosnia and Herzegovina')]
[('country', 'http://www.wikidata.org/entity/Q155'), ('countryName', 'Brazil')]
[('country', 'http://www.wikidata.org/entity/Q219'), ('countryName', 'Bulgaria')]
[('country', 'http://www.wikidata.org/entity/Q1009'), ('countryName', 'Cameroon')]
[('country', 'http://www.wikidata.org/entity/Q16'), ('countryName', 'Canada')]
[('c

In [94]:
# Now I'll add the count
queryString = """
SELECT DISTINCT ?countryName (COUNT(?country) AS ?apparitions)
WHERE {
   # bind something
   ?edition wdt:P3450 wd:Q19317 ;
       wdt:P1923 ?partecipatingTeam .
   {
       ?partecipatingTeam wdt:P1532 ?country .
   }
   UNION
   {
       ?partecipatingTeam wdt:P17 ?country .
       MINUS
       {
           ?partecipatingTeam ?p wd:Q145 .
       }
   }
   
   # get the label
   ?country sc:name ?countryName.
}
GROUP BY ?countryName
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('countryName', 'South Korea'), ('apparitions', '19')]
[('countryName', 'Slovenia'), ('apparitions', '4')]
[('countryName', 'Dutch East Indies'), ('apparitions', '1')]
[('countryName', 'Cameroon'), ('apparitions', '14')]
[('countryName', 'Poland'), ('apparitions', '15')]
[('countryName', 'Australia'), ('apparitions', '10')]
[('countryName', 'Uruguay'), ('apparitions', '26')]
[('countryName', 'Norway'), ('apparitions', '6')]
[('countryName', 'Czech Republic'), ('apparitions', '2')]
[('countryName', 'Cuba'), ('apparitions', '2')]
[('countryName', 'Algeria'), ('apparitions', '8')]
[('countryName', 'New Zealand'), ('apparitions', '4')]
[('countryName', 'Jamaica'), ('apparitions', '2')]
[('countryName', 'Iraq'), ('apparitions', '2')]
[('countryName', 'France'), ('apparitions', '29')]
[('countryName', 'Northern Ireland'), ('apparitions', '3')]
[('countryName', 'Chile'), ('apparitions', '18')]
[('countryName', 'Turkey'), ('apparitions', '4')]
[('countryName', 'Netherlands'), ('appari

In [89]:
queryString = """
SELECT DISTINCT ?country ?countryName (COUNT(?country) AS ?apparitions)
WHERE {
   # bind something
   ?edition wdt:P3450 wd:Q19317 ;
       wdt:P1923 ?partecipatingTeam .
   {
       ?partecipatingTeam wdt:P1532 ?country .
   }
   UNION
   {
       ?partecipatingTeam wdt:P17 ?country .
       MINUS
       {
           ?partecipatingTeam ?p wd:Q145 .
       }
   }
   
   # get the label
   ?country sc:name ?countryName.
}
GROUP BY ?country ?countryName
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q218'), ('countryName', 'Romania'), ('apparitions', '14')]
[('country', 'http://www.wikidata.org/entity/Q28'), ('countryName', 'Hungary'), ('apparitions', '18')]
[('country', 'http://www.wikidata.org/entity/Q974'), ('countryName', 'Democratic Republic of the Congo'), ('apparitions', '2')]
[('country', 'http://www.wikidata.org/entity/Q215'), ('countryName', 'Slovenia'), ('apparitions', '4')]
[('country', 'http://www.wikidata.org/entity/Q754'), ('countryName', 'Trinidad and Tobago'), ('apparitions', '2')]
[('country', 'http://www.wikidata.org/entity/Q38'), ('countryName', 'Italy'), ('apparitions', '36')]
[('country', 'http://www.wikidata.org/entity/Q16'), ('countryName', 'Canada'), ('apparitions', '2')]
[('country', 'http://www.wikidata.org/entity/Q664'), ('countryName', 'New Zealand'), ('apparitions', '4')]
[('country', 'http://www.wikidata.org/entity/Q3589884'), ('countryName', 'Egypt national under-20 football team'), ('apparitions'

In [95]:
# Now I'll add the having
queryString = """
SELECT DISTINCT ?country ?countryName (COUNT(?country) AS ?apparitions)
WHERE {
   # bind something
   ?edition wdt:P3450 wd:Q19317 ;
       wdt:P1923 ?partecipatingTeam .
   {
       ?partecipatingTeam wdt:P1532 ?country .
   }
   UNION
   {
       ?partecipatingTeam wdt:P17 ?country .
       MINUS
       {
           ?partecipatingTeam ?p wd:Q145 .
       }
   }
   
   # get the label
   ?country sc:name ?countryName.
}
GROUP BY ?country ?countryName
HAVING (COUNT(?country)>=10)
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q218'), ('countryName', 'Romania'), ('apparitions', '14')]
[('country', 'http://www.wikidata.org/entity/Q28'), ('countryName', 'Hungary'), ('apparitions', '18')]
[('country', 'http://www.wikidata.org/entity/Q38'), ('countryName', 'Italy'), ('apparitions', '36')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('countryName', 'Germany'), ('apparitions', '37')]
[('country', 'http://www.wikidata.org/entity/Q155'), ('countryName', 'Brazil'), ('apparitions', '42')]
[('country', 'http://www.wikidata.org/entity/Q31'), ('countryName', 'Belgium'), ('apparitions', '25')]
[('country', 'http://www.wikidata.org/entity/Q34'), ('countryName', 'Sweden'), ('apparitions', '24')]
[('country', 'http://www.wikidata.org/entity/Q419'), ('countryName', 'Peru'), ('apparitions', '10')]
[('country', 'http://www.wikidata.org/entity/Q29'), ('countryName', 'Spain'), ('apparitions', '30')]
[('country', 'http://www.wikidata.org/entity/Q414'), ('countryName', '

Final query for this task

In [96]:
# write your final query
queryString = """
SELECT DISTINCT ?country ?countryName (COUNT(?country) AS ?apparitions)
WHERE {
   # bind something
   ?edition wdt:P3450 wd:Q19317 ;
       wdt:P1923 ?partecipatingTeam .
   {
       ?partecipatingTeam wdt:P1532 ?country .
   }
   UNION
   {
       ?partecipatingTeam wdt:P17 ?country .
       MINUS
       {
           ?partecipatingTeam ?p wd:Q145 .
       }
   }
   
   # get the label
   ?country sc:name ?countryName.
}
GROUP BY ?country ?countryName
HAVING (COUNT(?country)>=10)
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q218'), ('countryName', 'Romania'), ('apparitions', '14')]
[('country', 'http://www.wikidata.org/entity/Q28'), ('countryName', 'Hungary'), ('apparitions', '18')]
[('country', 'http://www.wikidata.org/entity/Q38'), ('countryName', 'Italy'), ('apparitions', '36')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('countryName', 'Germany'), ('apparitions', '37')]
[('country', 'http://www.wikidata.org/entity/Q155'), ('countryName', 'Brazil'), ('apparitions', '42')]
[('country', 'http://www.wikidata.org/entity/Q31'), ('countryName', 'Belgium'), ('apparitions', '25')]
[('country', 'http://www.wikidata.org/entity/Q34'), ('countryName', 'Sweden'), ('apparitions', '24')]
[('country', 'http://www.wikidata.org/entity/Q419'), ('countryName', 'Peru'), ('apparitions', '10')]
[('country', 'http://www.wikidata.org/entity/Q29'), ('countryName', 'Spain'), ('apparitions', '30')]
[('country', 'http://www.wikidata.org/entity/Q414'), ('countryName', '

## Task 9
Consider the countries that won the FIFA World Cup at least twice. Find the association football clubs of those countries won the UEFA Champions League (the result set must be couples of clubs IRI and label).

In [97]:
# write your queries
# I'll start by retrieving all the winning P1346 teams of the FIFA World Cup
queryString = """
SELECT DISTINCT ?edition ?editionName
WHERE {
   # bind something
   ?edition wdt:P3450 wd:Q19317 ;
       wdt:P1346 ?national.
   # get the label
   ?edition sc:name ?editionName.
   ?national sc:name ?nationalName .
}
ORDER BY ?editionName
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('edition', 'http://www.wikidata.org/entity/Q48432'), ('editionName', '1930 FIFA World Cup')]
[('edition', 'http://www.wikidata.org/entity/Q1477177'), ('editionName', '1934 FIFA World Cup')]
[('edition', 'http://www.wikidata.org/entity/Q131591'), ('editionName', '1938 FIFA World Cup')]
[('edition', 'http://www.wikidata.org/entity/Q132515'), ('editionName', '1950 FIFA World Cup')]
[('edition', 'http://www.wikidata.org/entity/Q131075'), ('editionName', '1954 FIFA World Cup')]
[('edition', 'http://www.wikidata.org/entity/Q132533'), ('editionName', '1958 FIFA World Cup')]
[('edition', 'http://www.wikidata.org/entity/Q160813'), ('editionName', '1962 FIFA World Cup')]
[('edition', 'http://www.wikidata.org/entity/Q134202'), ('editionName', '1966 FIFA World Cup')]
[('edition', 'http://www.wikidata.org/entity/Q132664'), ('editionName', '1970 FIFA World Cup')]
[('edition', 'http://www.wikidata.org/entity/Q166121'), ('editionName', '1974 FIFA World Cup')]
[('edition', 'http://www.wikidat

In [98]:
queryString = """
SELECT DISTINCT ?edition ?editionName ?national ?nationalName
WHERE {
   # bind something
   ?edition wdt:P3450 wd:Q19317 ;
       wdt:P1346 ?national.
   # get the label
   ?edition sc:name ?editionName.
   ?national sc:name ?nationalName .
}
ORDER BY ?editionName
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('edition', 'http://www.wikidata.org/entity/Q48432'), ('editionName', '1930 FIFA World Cup'), ('national', 'http://www.wikidata.org/entity/Q134916'), ('nationalName', 'Uruguay national football team')]
[('edition', 'http://www.wikidata.org/entity/Q1477177'), ('editionName', '1934 FIFA World Cup'), ('national', 'http://www.wikidata.org/entity/Q676899'), ('nationalName', 'Italy national association football team')]
[('edition', 'http://www.wikidata.org/entity/Q131591'), ('editionName', '1938 FIFA World Cup'), ('national', 'http://www.wikidata.org/entity/Q676899'), ('nationalName', 'Italy national association football team')]
[('edition', 'http://www.wikidata.org/entity/Q132515'), ('editionName', '1950 FIFA World Cup'), ('national', 'http://www.wikidata.org/entity/Q134916'), ('nationalName', 'Uruguay national football team')]
[('edition', 'http://www.wikidata.org/entity/Q131075'), ('editionName', '1954 FIFA World Cup'), ('national', 'http://www.wikidata.org/entity/Q43310'), ('nat

In [99]:
# for each team, I'll retrieve its country.
# I'll start by looking at the predicates and objs of the Q676899 Italy national association football team
queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName
WHERE {
   # bind something
   wd:Q676899 ?p ?obj .
   # get the label
   ?p sc:name ?pName.
   ?obj sc:name ?objName .
}
ORDER BY ?editionName
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('pName', 'participant in'), ('obj', 'http://www.wikidata.org/entity/Q63865'), ('objName', 'UEFA Euro 2000 Final')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('pName', 'participant in'), ('obj', 'http://www.wikidata.org/entity/Q101751'), ('objName', '1994 FIFA World Cup')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('pName', 'participant in'), ('obj', 'http://www.wikidata.org/entity/Q131591'), ('objName', '1938 FIFA World Cup')]
[('p', 'http://www.wikidata.org/prop/direct/P2522'), ('pName', 'victory'), ('obj', 'http://www.wikidata.org/entity/Q131591'), ('objName', '1938 FIFA World Cup')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('pName', 'participant in'), ('obj', 'http://www.wikidata.org/entity/Q132664'), ('objName', '1970 FIFA World Cup')]
[('p', 'http://www.wikidata.org/prop/direct/P463'), ('pName', 'member of'), ('obj', 'http://www.wikidata.org/entity/Q253414'), ('objName', 'FIFA')]
[('p',

In [103]:
# as for task 8, I'll retrieve the country using the union on wdt:P1532 and wdt:P17 
queryString = """
SELECT DISTINCT ?p ?pName ?country ?countryName
WHERE {
   # bind something
   ?edition wdt:P3450 wd:Q19317 ;
       wdt:P1346 ?national.
   {
      ?national wdt:P1532 ?country .
   }
   UNION
   {
      ?national wdt:P17 ?country .
   }
   # get the label
   ?country sc:name ?countryName.
   ?edition sc:name ?editionName.
}
ORDER BY ?editionName
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q77'), ('countryName', 'Uruguay')]
[('country', 'http://www.wikidata.org/entity/Q38'), ('countryName', 'Italy')]
[('country', 'http://www.wikidata.org/entity/Q21'), ('countryName', 'England')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('countryName', 'United Kingdom')]
[('country', 'http://www.wikidata.org/entity/Q414'), ('countryName', 'Argentina')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France')]
[('country', 'http://www.wikidata.org/entity/Q155'), ('countryName', 'Brazil')]
[('country', 'http://www.wikidata.org/entity/Q29'), ('countryName', 'Spain')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('countryName', 'Germany')]
9


In [104]:
# as for task 8, I have to subtract the United Kingdom country
queryString = """
SELECT DISTINCT ?edition ?editionName ?country ?countryName
WHERE {
   # bind something
   ?edition wdt:P3450 wd:Q19317 ;
       wdt:P1346 ?national.
   {
      ?national wdt:P1532 ?country .
   }
   UNION
   {
      ?national wdt:P17 ?country .
      MINUS
      {
          ?country ?p wd:Q145 .
      }
   }
   # get the label
   ?country sc:name ?countryName.
   ?edition sc:name ?editionName.
}
ORDER BY ?editionName
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('edition', 'http://www.wikidata.org/entity/Q48432'), ('editionName', '1930 FIFA World Cup'), ('country', 'http://www.wikidata.org/entity/Q77'), ('countryName', 'Uruguay')]
[('edition', 'http://www.wikidata.org/entity/Q1477177'), ('editionName', '1934 FIFA World Cup'), ('country', 'http://www.wikidata.org/entity/Q38'), ('countryName', 'Italy')]
[('edition', 'http://www.wikidata.org/entity/Q131591'), ('editionName', '1938 FIFA World Cup'), ('country', 'http://www.wikidata.org/entity/Q38'), ('countryName', 'Italy')]
[('edition', 'http://www.wikidata.org/entity/Q132515'), ('editionName', '1950 FIFA World Cup'), ('country', 'http://www.wikidata.org/entity/Q77'), ('countryName', 'Uruguay')]
[('edition', 'http://www.wikidata.org/entity/Q131075'), ('editionName', '1954 FIFA World Cup'), ('country', 'http://www.wikidata.org/entity/Q183'), ('countryName', 'Germany')]
[('edition', 'http://www.wikidata.org/entity/Q132533'), ('editionName', '1958 FIFA World Cup'), ('country', 'http://www.

In [106]:
# now I'll group by country, count and filter using having
queryString = """
SELECT DISTINCT ?country ?countryName
WHERE {
   # bind something
   ?edition wdt:P3450 wd:Q19317 ;
       wdt:P1346 ?national.
   {
      ?national wdt:P1532 ?country .
   }
   UNION
   {
      ?national wdt:P17 ?country .
      MINUS
      {
          ?country ?p wd:Q145 .
      }
   }
   # get the label
   ?country sc:name ?countryName.
   ?edition sc:name ?editionName.
}
GROUP BY ?country ?countryName
HAVING (COUNT(?country)>=2)
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q38'), ('countryName', 'Italy')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('countryName', 'Germany')]
[('country', 'http://www.wikidata.org/entity/Q155'), ('countryName', 'Brazil')]
[('country', 'http://www.wikidata.org/entity/Q414'), ('countryName', 'Argentina')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France')]
[('country', 'http://www.wikidata.org/entity/Q77'), ('countryName', 'Uruguay')]
6


In [108]:
# Now I'll select all the winners of the UCL from the countries retrieved from previous query 
queryString = """
SELECT DISTINCT ?country ?countryName
WHERE {
   # bind something
   ?edition wdt:P3450 wd:Q19317 ;
       wdt:P1346 ?national.
   {
      ?national wdt:P1532 ?country .
   }
   UNION
   {
      ?national wdt:P17 ?country .
      MINUS
      {
          ?country ?p wd:Q145 .
      }
   }
   # bind something
   ?UCLEdition wdt:P3450 wd:Q18756 ;
           wdt:P1346 ?winner .
   ?winner wdt:P17 ?country .
   # get the label
   ?UCLEdition sc:name ?UCLEditionName .
   ?winner sc:name ?winnerName .
    
   # get the label
   ?country sc:name ?countryName.
   ?edition sc:name ?editionName.
}
GROUP BY ?country ?countryName
HAVING (COUNT(?country)>=2)
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q38'), ('countryName', 'Italy')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('countryName', 'Germany')]
[('country', 'http://www.wikidata.org/entity/Q29'), ('countryName', 'Spain')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France')]
4


In [114]:
# previous query was obviously wrong. I need a subquery in order to get the result.
# Now I'll match the country of the winners of UCL with the countries of the subquery, that gives the countries that won the FIFA World cup at least 2 times.
queryString = """
SELECT DISTINCT ?club ?clubName
WHERE{
   #bind something
   ?UCLEdition wdt:P3450 wd:Q18756 ;
           wdt:P1346 ?club .
   ?club wdt:P17 ?country .
   # get the label
   ?club sc:name ?clubName .
   {
    SELECT DISTINCT ?country ?countryName
    WHERE {
       # bind something
       ?edition wdt:P3450 wd:Q19317 ;
           wdt:P1346 ?national.
       {
          ?national wdt:P1532 ?country .
       }
       UNION
       {
          ?national wdt:P17 ?country .
          MINUS
          {
              ?country ?p wd:Q145 .
          }
       }
       # get the label
       ?country sc:name ?countryName.
    }
    GROUP BY ?country ?countryName
    HAVING (COUNT(?country)>=2)
    }
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('club', 'http://www.wikidata.org/entity/Q132885'), ('clubName', 'Olympique de Marseille')]
[('club', 'http://www.wikidata.org/entity/Q15789'), ('clubName', 'FC Bayern Munich')]
[('club', 'http://www.wikidata.org/entity/Q41420'), ('clubName', 'Borussia Dortmund')]
[('club', 'http://www.wikidata.org/entity/Q51974'), ('clubName', 'Hamburger SV')]
[('club', 'http://www.wikidata.org/entity/Q1422'), ('clubName', 'Juventus F.C.')]
[('club', 'http://www.wikidata.org/entity/Q1543'), ('clubName', 'A.C. Milan')]
[('club', 'http://www.wikidata.org/entity/Q631'), ('clubName', 'FC Internazionale Milano')]
7


Final query for this task

In [117]:
# write your final query
queryString = """
SELECT DISTINCT ?club ?clubName
WHERE{
   #bind something
   ?UCLEdition wdt:P3450 wd:Q18756 ;
           wdt:P1346 ?club .
   ?club wdt:P17 ?country .
   # get the label
   ?club sc:name ?clubName .
   {
    SELECT DISTINCT ?country ?countryName
    WHERE {
       # bind something
       ?edition wdt:P3450 wd:Q19317 ;
           wdt:P1346 ?national.
       {
          ?national wdt:P1532 ?country .
       }
       UNION
       {
          ?national wdt:P17 ?country .
          MINUS
          {
              ?country ?p wd:Q145 .
          }
       }
       # get the label
       ?country sc:name ?countryName.
    }
    GROUP BY ?country ?countryName
    HAVING (COUNT(?country)>=2)
    }
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('club', 'http://www.wikidata.org/entity/Q132885'), ('clubName', 'Olympique de Marseille')]
[('club', 'http://www.wikidata.org/entity/Q15789'), ('clubName', 'FC Bayern Munich')]
[('club', 'http://www.wikidata.org/entity/Q41420'), ('clubName', 'Borussia Dortmund')]
[('club', 'http://www.wikidata.org/entity/Q51974'), ('clubName', 'Hamburger SV')]
[('club', 'http://www.wikidata.org/entity/Q1422'), ('clubName', 'Juventus F.C.')]
[('club', 'http://www.wikidata.org/entity/Q1543'), ('clubName', 'A.C. Milan')]
[('club', 'http://www.wikidata.org/entity/Q631'), ('clubName', 'FC Internazionale Milano')]
7
